## 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time , datetime


## 定义数据库连接

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

## 导入租后数据

In [3]:
# f_path_ck = "F:/myfile/租后数据/租后应收监控excel2024-07-24.xlsx"

# df_zhys = pd.read_excel(f_path_ck,sheet_name='租后应收监控')

# df_zhys.shape

In [4]:
# df_ddfq = pd.read_excel(f_path_ck,sheet_name='订单分期表')
# df_ddfq.shape

In [5]:
# df_zh = df_ddfq.merge(df_zhys,on = '订单号',how = 'left' )
# df_zh.shape

# 获取租后数据

In [6]:
sql = ''' -- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,om.order_tag
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- WHERE tprm.order_create_time >= DATE_ADD(CURRENT_DATE, INTERVAL -11 DAY)
-- and tprm.order_create_time <= DATE_ADD(CURRENT_DATE, INTERVAL -4 DAY)
;
'''

df_zhys = query(sql)
df_zhys.shape

(27787, 62)

In [7]:
df_zhys.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [8]:
pds = pd.ExcelFile('F:/myfile/业务相关数据/iPhone最新采购价/9月24日iPhone15-16报价1.xlsx')
pds.sheet_names
iPhone15 = pds.parse('15系列')
iPhone15['商品名称'] = iPhone15['商品名称'].ffill().str.replace(' ', '')
iPhone15['型号颜色'] = iPhone15['标题'].str.split('/A  ').apply(lambda x: x[1] if len(x) > 1 else None).str.replace(' ', '').str.replace('B', '')


iPhone16 = pds.parse('新款16系列')
iPhone16 = iPhone16.ffill()
iPhone16['商品名称'] = iPhone16['商品名称'].ffill().str.replace(' ', '')
iPhone16['型号颜色'] = iPhone16['标题'].str.split('/A  ').apply(lambda x: x[1] if len(x) > 1 else None).str.replace(' ', '').str.replace('B', '')

iPhone = pd.concat([iPhone15, iPhone16])
iPhone = iPhone[['商品名称','型号颜色', '含税报价']]
# iPhone['商品名称'].drop_duplicates()
# i_list = iPhone['型号颜色'].to_list()
iPhone

,商品名称,型号颜色,含税报价
0,iPhone15,iPhone15128G黑色,4925
1,iPhone15,iPhone15128G粉色,4925
2,iPhone15,iPhone15128G蓝色,4950
3,iPhone15,iPhone15128G绿色,4920
4,iPhone15,iPhone15256G黑色,5930
...,...,...,...
23,iPhone16ProMax,iPhone16ProMax512G沙漠色,12295
24,iPhone16ProMax,iPhone16ProMax1T黑色,14440
25,iPhone16ProMax,iPhone16ProMax1T白色,14690
26,iPhone16ProMax,iPhone16ProMax1T原色,14650


In [9]:
df_zhys.shape

(27787, 62)

In [10]:
df_zhys = df_zhys[df_zhys['rent_received']!=0]
df_zhys.shape

(27509, 62)

In [11]:

# df_zhys[(df_zhys['rent_received']==630.25) & (df_zhys['paid_buyout_balance_payment']==12366.45)]['total_receivable2']

In [12]:
def ygcz(row):
    if pd.isna(row['product_name']):
        return row['采购金额2'] * 0.5
    elif 'iPhone 16' in row['product_name']:
        return row['采购金额2'] * 0.6
    elif 'iPhone15' in row['product_name']:
        return row['采购金额2'] * 0.4
    else:
        return row['采购金额2'] * 0.425


In [13]:
def new_price(x):
    if pd.isna(x):
        return None
    if 'iPhone15' in x and '128G' in x and  '黑色' in x:
        return 4925
    if 'iPhone15' in x and '128G' in x and  '粉色' in x:
        return 4925
    if 'iPhone15' in x and '128G' in x and  '蓝色' in x:
        return 4950
    if 'iPhone15' in x and '128G' in x and  '绿色' in x:
        return 4920
        
    if 'iPhone15' in x and '256G' in x and  '黑色' in x:
        return 5930
    if 'iPhone15' in x and '256G' in x and  '粉色' in x:
        return 5930
    if 'iPhone15' in x and '256G' in x and  '蓝色' in x:
        return 5975
    if 'iPhone15' in x and '256G' in x and  '绿色' in x:
        return 5850

    if 'iPhone15' in x and '512G' in x and  '黑色' in x:
        return 8070
    if 'iPhone15' in x and '512G' in x and  '粉色' in x:
        return 8070
    if 'iPhone15' in x and '512G' in x and  '蓝色' in x:
        return 8090
    if 'iPhone15' in x and '512G' in x and  '绿色' in x:
        return 8090

# MTX93CH/A  iPhone 15 Plus 128GB 黑色	6999	5430
# MTXA3CH/A  iPhone 15 Plus 128GB 粉色	6999	5470
# MTXD3CH/A  iPhone 15 Plus 128GB 蓝色	6999	5490
# MTXE3CH/A  iPhone 15 Plus 128GB 绿色	6999	5490
    
# MTXF3CH/A  iPhone 15 Plus 256GB 黑色	7999	6415
# MTXG3CH/A  iPhone 15 Plus 256GB 粉色	7999	6425
# MTXJ3CH/A  iPhone 15 Plus 256GB 蓝色	7999	6500
# MTXK3CH/A  iPhone 15 Plus 256GB 绿色	7999	6500



    if 'iPhone15Plus' in x and '128G' in x and  '黑色' in x:
        return 5430
    if 'iPhone15Plus' in x and '128G' in x and  '粉色' in x:
        return 5470
    if 'iPhone15Plus' in x and '128G' in x and  '蓝色' in x:
        return 5490
    if 'iPhone15Plus' in x and '128G' in x and  '绿色' in x:
        return 5490
        
    if 'iPhone15Plus' in x and '256G' in x and  '黑色' in x:
        return 6415
    if 'iPhone15Plus' in x and '256G' in x and  '粉色' in x:
        return 6425
    if 'iPhone15Plus' in x and '256G' in x and  '蓝色' in x:
        return 6500
    if 'iPhone15Plus' in x and '256G' in x and  '绿色' in x:
        return 6500

# MTQ43CH/A  iPhone 15 Pro 128GB 黑色钛金属	7999	6690
# MTQ53CH/A  iPhone 15 Pro 128GB 白色钛金属	7999	6690
# MTQ63CH/A  iPhone 15 Pro 128GB 原色钛金属	7999	6730
# MTQ73CH/A  iPhone 15 Pro 128GB 蓝色钛金属	7999	6580
    
# MTQ83CH/A  iPhone 15 Pro 256GB 黑色钛金属	8999	7530
# MTQ93CH/A  iPhone 15 Pro 256GB 白色钛金属	8999	7510
# MTQA3CH/A  iPhone 15 Pro 256GB 原色钛金属	8999	7470
# MTQC3CH/A  iPhone 15 Pro 256GB 蓝色钛金属	8999	7530
    
# MTQD3CH/A  iPhone 15 Pro 512GB 黑色钛金属	10999	9070
# MTQE3CH/A  iPhone 15 Pro 512GB 白色钛金属	10999	9070
# MTQF3CH/A  iPhone 15 Pro 512GB 原色钛金属	10999	9070
# MTQG3CH/A  iPhone 15 Pro 512GB 蓝色钛金属	10999	9090
    
# MTQH3CH/A  iPhone 15 Pro 1TB 黑色钛金属	12999	9830
# MTQJ3CH/A  iPhone 15 Pro 1TB 白色钛金属	12999	9830
# MTQK3CH/A  iPhone 15 Pro 1TB 原色钛金属	12999	9830
# MTQL3CH/A  iPhone 15 Pro 1TB 蓝色钛金属	12999	9830
    if 'iPhone15Pro' in x and '128G' in x and  '黑色钛金属' in x:
        return 6690
    if 'iPhone15Pro' in x and '128G' in x and  '白色钛金属' in x:
        return 6690
    if 'iPhone15Pro' in x and '128G' in x and  '原色钛金属' in x:
        return 6730
    if 'iPhone15Pro' in x and '128G' in x and  '蓝色钛金属' in x:
        return 6580
        
    if 'iPhone15Pro' in x and '256G' in x and  '黑色钛金属' in x:
        return 7530
    if 'iPhone15Pro' in x and '256G' in x and  '白色钛金属' in x:
        return 7510
    if 'iPhone15Pro' in x and '256G' in x and  '原色钛金属' in x:
        return 7470
    if 'iPhone15Pro' in x and '256G' in x and  '蓝色钛金属' in x:
        return 7530

    if 'iPhone15Pro' in x and '512G' in x and  '黑色钛金属' in x:
        return 9070
    if 'iPhone15Pro' in x and '512G' in x and  '白色钛金属' in x:
        return 9070
    if 'iPhone15Pro' in x and '512G' in x and  '原色钛金属' in x:
        return 9070
    if 'iPhone15Pro' in x and '512G' in x and  '蓝色钛金属' in x:
        return 9090

# MU2N3CH/A  iPhone 15 Pro Max 256GB 黑色钛金属	9999	8320
# MU2P3CH/A  iPhone 15 Pro Max 256GB 白色钛金属	9999	8300
# MU2Q3CH/A  iPhone 15 Pro Max 256GB 原色钛金属	9999	8300
# MU2R3CH/A  iPhone 15 Pro Max 256GB 蓝色钛金属	9999	8300

# MU2T3CH/A  iPhone 15 Pro Max 512GB 黑色钛金属	11999	9855
# MU2U3CH/A  iPhone 15 Pro Max 512GB 白色钛金属	11999	9855
# MU2V3CH/A  iPhone 15 Pro Max 512GB 原色钛金属	11999	9855
# MU2W3CH/A  iPhone 15 Pro Max 512GB 蓝色钛金属	11999	9855

# MU2X3CH/A  iPhone 15 Pro Max 1TB 黑色钛金属	13999	11500
# MU2Y3CH/A  iPhone 15 Pro Max 1TB 白色钛金属	13999	11500
# MU603CH/A  iPhone 15 Pro Max 1TB 原色钛金属	13999	11500
    if 'iPhone15ProMax' in x and '256G' in x and  '黑色钛金属' in x:
        return 8320
    if 'iPhone15ProMax' in x and '256G' in x and  '白色钛金属' in x:
        return 8300
    if 'iPhone15ProMax' in x and '256G' in x and  '原色钛金属' in x:
        return 8300
    if 'iPhone15ProMax' in x and '256G' in x and  '蓝色钛金属' in x:
        return 8300
        
    if 'iPhone15ProMax' in x and '512G' in x and  '黑色钛金属' in x:
        return 9855
    if 'iPhone15ProMax' in x and '512G' in x and  '白色钛金属' in x:
        return 9855
    if 'iPhone15ProMax' in x and '512G' in x and  '原色钛金属' in x:
        return 9855
    if 'iPhone15ProMax' in x and '512G' in x and  '蓝色钛金属' in x:
        return 9855

# MTEY3CH/A  iPhone 16 128GB 群青色	5999	5930
# MTF43CH/A  iPhone 16 256GB 群青色	6999	6950
# MTF73CH/A  iPhone 16 512GB 白色	8999	8880
# MTF83CH/A  iPhone 16 512GB 粉色	8999	8780
# MTF93CH/A  iPhone 16 512GB 群青色	8999	8830
# MTFA3CH/A  iPhone 16 512GB 深青色	8999	8830   iPhone 16 Pro
    if 'iPhone16' in x and '128G' in x and  '群青色' in x:
        return 5930
    if 'iPhone16' in x and '256G' in x and  '群青色' in x:
        return 6950
    if 'iPhone16' in x and '512G' in x and  '白色' in x:
        return 8880
    if 'iPhone16' in x and '512G' in x and  '粉色' in x:
        return 8780
    if 'iPhone16' in x and '512G' in x and  '群青色' in x:
        return 8830
    if 'iPhone16' in x and '512G' in x and  '深青色' in x:
        return 8830

# MTUD3CH/A  iPhone 16 Plus 128GB 群青色	6999	6940
# MTUF3CH/A  iPhone 16 Plus 256GB 黑色	7999	7910
# MTUH3CH/A  iPhone 16 Plus 256GB 粉色	7999	7910
# MTUJ3CH/A  iPhone 16 Plus 256GB 群青色	7999	7910
# MTUQ3CH/A  iPhone 16Plus 512GB 群青色	9999	9750
# MTLR3CH/A  iPhone 16 Pro 128GB 原色	7999	8220
# MTLQ3CH/A  iPhone 16 Pro 128GB沙漠色	7999	8220
    if 'iPhone16Plus' in x and '128G' in x and  '群青色' in x:
        return 6940
    if 'iPhone16Plus' in x and '256G' in x and  '黑色' in x:
        return 7910
    if 'iPhone16Plus' in x and '256G' in x and  '粉色' in x:
        return 7910
    if 'iPhone16Plus' in x and '256G' in x and  '群青色' in x:
        return 7910
    if 'iPhone16Plus' in x and '512G' in x and  '群青色' in x:
        return 9750
    if 'iPhone16Pro' in x and '128G' in x and  '原色' in x:
        return 8220
    if 'iPhone16Pro' in x and '128G' in x and  '沙漠色' in x:
        return 8220

# MTLT3CH/A  iPhone 16 Pro 256GB 黑色	8999	9190
# MTLW3CH/A  iPhone 16 Pro 256GB 原色	8999	9130
# MTLX3CH/A  iPhone 16 Pro 512GB 黑色	10999	11380
# MTM43CH/A  iPhone 16 Pro 512GB 原色	10999	11170

    if 'iPhone16Pro' in x and '256G' in x and  '黑色钛金属' in x:
        return 9190
    if 'iPhone16Pro' in x and '256G' in x and  '原色钛金属' in x:
        return 9130
    if 'iPhone16Pro' in x and '512G' in x and  '黑色钛金属' in x:
        return 11380
    if 'iPhone16Pro' in x and '512G' in x and  '原色钛金属' in x:
        return 11170

# MUTR3CH/A  iPhone 16 Pro Max 512GB 黑色	11999	12250
# MUTX3CH/A  iPhone 16 Pro Max 512GB 原色	11999	12295
# MUTW3CH/A  iPhone 16 Pro Max 512GB 沙漠色	11999	12295

    if 'iPhone16ProMax' in x and '512G' in x and  '黑色钛金属' in x:
        return 12250
    if 'iPhone16ProMax' in x and '512G' in x and  '白色钛金属' in x:
        return 12295
    if 'iPhone16ProMax' in x and '512G' in x and  '沙漠色钛金属 ' in x:
        return 12295
    return None  # 默认返回值，处理未匹配的情况
 
        

In [14]:
df_zhys['product_name'] = df_zhys['model_number']+df_zhys['specification']

In [ ]:

df_zhys['下单日期'] = df_zhys['order_create_time'].dt.date
df_zhys["内存"]=df_zhys["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
df_zhys["颜色"]=df_zhys["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
df_zhys['型号颜色1'] = df_zhys['model_number'].str.cat([df_zhys['内存'], df_zhys['颜色']], sep=' ').str.replace(' ', '')

df_zhys['型号颜色1'] = df_zhys['型号颜色1'].dropna()
# 使用 apply 函数更新采购金额
df_zhys['采购金额2'] = df_zhys['product_name'].apply(new_price)
# df_zhys.采购金额2.value_counts()
# 如果需要保留旧采购金额，可以选择将新值与旧值结合
df_zhys['采购金额2'] = df_zhys['采购金额2'].fillna(df_zhys['purchase_amount'])
df_zhys = df_zhys[df_zhys['采购金额2']!=0]

In [16]:
# df_zhys[['buyout_balance_payment', '预估残值2', 'product_name', '采购金额2']]

In [17]:
# df_zhys[['buyout_balance_payment', '预估残值2', 'product_name', '采购金额2']].to_excel('F:/ttt.xlsx')

In [18]:
df_zhys['product_name'].info()

<class 'pandas.core.series.Series'>
Index: 25902 entries, 0 to 27784
Series name: product_name
Non-Null Count  Dtype 
--------------  ----- 
25902 non-null  object
dtypes: object(1)
memory usage: 404.7+ KB


In [19]:


df_zhys['预估残值2'] = df_zhys.apply(ygcz, axis=1)

df_zhys['总已收2'] = df_zhys['rent_received'] + np.where(df_zhys['paid_buyout_balance_payment']==0, df_zhys['预估残值2'], df_zhys['paid_buyout_balance_payment'])


df_zhys

# df_zhys[['型号颜色1', 'purchase_amount', '采购金额2', '预估残值2', '总已收2', 'rent_received', 'paid_buyout_balance_payment']].to_excel('F:/ts1.xlsx')

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,...,mobile,order_tag,product_name,下单日期,内存,颜色,型号颜色1,采购金额2,预估残值2,总已收2
0,1,8924,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,...,13027948681,0.0,99新 iPhone 12 Pro 国行 官方质检【租物特惠】内存：128G 颜色：金色,2022-07-05,128G,金色,99新iPhone12Pro国行官方质检【租物特惠】128G金色,5150.0,2188.75,8176.92
2,3,9514,A202207151707153,2022年07月,2022-07,2023-07-15,14,迪瓜优选,0,未知渠道,...,15297728428,0.0,全新vivo X80 Pro 高通骁龙8蔡司智能5G【租物特惠】内存：12GB+256G 颜...,2022-07-15,12GB+256G,假日,全新vivoX80Pro高通骁龙8蔡司智能5G【租物特惠】12GB+256G假日,4794.0,2037.45,7398.07
3,4,9600,A202207162356303,2022年07月,2022-07,2024-06-16,14,迪瓜优选,0,未知渠道,...,17643203514,1.0,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】内存：256G 颜色...,2022-07-16,256G,苍岭绿色,全新苹果iPhone13ProMax国行全网通【租物特惠】256G苍岭绿色,9180.0,3901.50,13146.80
4,5,9683,A202207171820512,2022年07月,2022-07,2023-07-17,14,迪瓜优选,0,未知渠道,...,13750296584,0.0,vivo Y55s 5G手机 6000mAh巨能量电池【租物特惠】内存：8GB+128GB ...,2022-07-17,8GB+128GB,樱粉流星,vivoY55s5G手机6000mAh巨能量电池【租物特惠】8GB+128GB樱粉流星,1214.0,515.95,2208.00
5,6,9699,A202207172051293,2022年07月,2022-07,2023-07-17,14,迪瓜优选,0,未知渠道,...,17607570936,NaN,vivo X80 1元租 高通骁龙8蔡司智能5G【活动专属】内存：12GB+256G 颜色：...,2022-07-17,12GB+256G,假日,vivoX801元租高通骁龙8蔡司智能5G【活动专属】12GB+256G假日,3478.0,1478.15,5719.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27779,27780,1765838,A2024112417193870,2024年11月,2024-11,2025-11-27,140,西安和兴德创,3,顶部搜索框的搜索结果页,...,17751375543,0.0,iPhone16ProMax内存：256GB 颜色：黑色钛金属,2024-11-24,256GB,黑色钛金属,iPhone16ProMax256GB黑色钛金属,9190.0,3905.75,4950.25
27780,27781,1765875,A20241124173159107,2024年11月,2024-11,2025-11-27,139,博威世纪科技,3,顶部搜索框的搜索结果页,...,15255380930,0.0,iPhone15内存：256G 颜色：粉色,2024-11-24,256G,粉色,iPhone15256G粉色,5930.0,2372.00,3064.99
27781,27782,1765899,A20241124173600131,2024年11月,2024-11,2025-11-27,139,博威世纪科技,20,其他渠道场景渠道。,...,18643435053,0.0,iPhone16ProMax内存：256GB 颜色：原色钛金属,2024-11-24,256GB,原色钛金属,iPhone16ProMax256GB原色钛金属,9130.0,3880.25,4575.46
27782,27783,1765950,A20241124175357182,2024年11月,2024-11,2025-11-27,140,西安和兴德创,12,芝麻信用频道,...,13073445971,0.0,iPhone16ProMax内存：256GB 颜色：原色钛金属,2024-11-24,256GB,原色钛金属,iPhone16ProMax256GB原色钛金属,9130.0,3880.25,4398.53


In [20]:
# df_zhys[df_zhys['rent_received']==0][['采购金额2', '预估残值2', 'rent_received', 'paid_buyout_balance_payment', '总已收2', 'gross_profit_receivable2']]

In [21]:
df_zhys['total_receivable2'] = df_zhys['all_rental'] + df_zhys['预估残值2']

In [22]:
status1_list = ['已逾期', '已买断', '已完成']
status2_list = ['还款中', '续租中', '未到首期还款日']
# df_zhys['实际毛利2'] = np.where(df_zhys['rembursement_status'].isin(status1_list), df_zhys['总已收2']-df_zhys['采购金额2'], df_zhys['actual_gross_profit'])
# df_zhys['实际毛利2'] = np.where(df_zhys['rembursement_status'].isin(status2_list), df_zhys['total_receivable2']-df_zhys['采购金额2'], df_zhys['actual_gross_profit'])
df_zhys['实际毛利2'] = np.where(df_zhys['rembursement_status'].isin(status2_list), df_zhys['total_receivable2']-df_zhys['采购金额2'], 
                            np.where(df_zhys['rembursement_status'].isin(status1_list), df_zhys['总已收2']-df_zhys['采购金额2'], df_zhys['actual_gross_profit']))
# dff = df_zhys[['型号颜色1', 'purchase_amount', '采购金额2', '预估残值2', '总已收2', 'rent_received', 'paid_buyout_balance_payment', 
#          '实际毛利2', 'rembursement_status', 'total_receivable', 'actual_gross_profit']]
df_zhys['gross_profit_receivable2'] = df_zhys['total_receivable2'] - df_zhys['采购金额2']

In [23]:
df_zhys['rembursement_status'].value_counts()

rembursement_status
还款中        12773
已逾期         5492
已买断         2406
未到首期还款日     2251
续租中         2135
已完成          772
检测中           73
Name: count, dtype: int64

In [24]:
## 吴总需求   每 1-2 周
# 下单月份	订单数	采购金额	总押金	总租金	预估残值	总应收	已收租金	未收租金	已归还残值/已付买断尾款	总已收	总未收	未回收成本	逾期总未收	应收逾期率	逾期损失	成本逾期率	
# 应收毛利	应收毛利率(应收毛利/采购成本）	实际毛利	"毛利率 (实际毛利/采购成本）"

# mon_zhys = df_zhys.groupby(['search_time']).agg({'order_number':'size','purchase_amount':'sum','all_deposit':'sum','all_rental':'sum','buyout_balance_payment':'sum','total_receivable':'sum'
#                                       ,'rent_received':'sum','unearned_rent_revenue':'sum','paid_buyout_balance_payment':'sum','total_received':'sum','total_unreceived':'sum'
#                                       ,'unrecovered_cost':'sum','overdue_total_unreceived':'sum','overdue_loss':'sum','gross_profit_receivable':'sum','actual_gross_profit':'sum'
#                                     })
mon_zhys = df_zhys.groupby(['下单日期']).agg({'order_number':'size','purchase_amount':'sum','all_deposit':'sum','all_rental':'sum','buyout_balance_payment':'sum','total_receivable2':'sum'
                                      ,'rent_received':'sum','unearned_rent_revenue':'sum','paid_buyout_balance_payment':'sum','total_received':'sum','total_unreceived':'sum'
                                      ,'unrecovered_cost':'sum','overdue_total_unreceived':'sum','overdue_loss':'sum','gross_profit_receivable2':'sum','actual_gross_profit':'sum'
                                      ,'采购金额2':'sum','预估残值2':'sum', '总已收2': 'sum', '实际毛利2': 'sum'
                                    })
mon_zhys['overdue_rate'] =   mon_zhys.overdue_total_unreceived/mon_zhys.total_receivable2    
mon_zhys['loss_rate'] =   mon_zhys.overdue_loss/mon_zhys.purchase_amount    
mon_zhys['receivable_profite_rate'] =   mon_zhys.gross_profit_receivable2/mon_zhys.purchase_amount    
mon_zhys['profite_rate'] =   mon_zhys.actual_gross_profit/mon_zhys.purchase_amount    


mon_zhys = mon_zhys[['order_number','purchase_amount','采购金额2', 'all_deposit','all_rental','buyout_balance_payment','预估残值2', 'total_receivable2','rent_received','unearned_rent_revenue','paid_buyout_balance_payment', '总已收2', 'total_received','total_unreceived'
                                      ,'unrecovered_cost','overdue_total_unreceived','overdue_rate','overdue_loss','loss_rate','gross_profit_receivable2','receivable_profite_rate','actual_gross_profit', '实际毛利2', 'profite_rate']]


In [25]:
mon_zhys2 = mon_zhys.reset_index()
mon_zhys2.head()

,下单日期,order_number,purchase_amount,采购金额2,all_deposit,all_rental,buyout_balance_payment,预估残值2,total_receivable2,rent_received,...,unrecovered_cost,overdue_total_unreceived,overdue_rate,overdue_loss,loss_rate,gross_profit_receivable2,receivable_profite_rate,actual_gross_profit,实际毛利2,profite_rate
0,2022-07-05,1,5150.0,5150.0,5303.0,4295.40,3181.80,2188.75,6484.15,7847.20,...,-3026.92,0.00,0.000000,0.00,0.000000,1334.15,0.259058,3026.92,3026.92,0.587751
1,2022-07-15,1,4794.0,4794.0,5999.0,4799.16,2549.57,2037.45,6836.61,4799.16,...,-2604.07,0.00,0.000000,0.00,0.000000,2042.61,0.426076,2604.07,2604.07,0.543194
2,2022-07-16,1,9180.0,9180.0,9799.0,6859.20,5879.40,3901.50,10760.70,6859.20,...,-3966.80,0.00,0.000000,0.00,0.000000,1580.70,0.172190,3966.80,3966.80,0.432113
3,2022-07-17,2,4692.0,4692.0,6098.0,4929.69,2591.65,1994.10,6923.79,4929.69,...,-3235.00,0.00,0.000000,0.00,0.000000,2231.79,0.475659,3235.00,3235.00,0.689471
4,2022-07-18,2,6678.0,6678.0,7749.0,6232.80,3879.58,2838.15,9070.95,4699.80,...,-65.24,3402.57,0.375106,1491.76,0.223384,2392.95,0.358333,65.24,1543.39,0.009769


In [26]:
# # 下单月份	订单数	采购金额	总押金	总租金	预估残值	总应收	已收租金	未收租金	已归还残值/已付买断尾款	总已收	总未收	未回收成本	逾期总未收	应收逾期率	逾期损失	成本逾期率	
# 应收毛利	应收毛利率(应收毛利/采购成本）	实际毛利	"毛利率 (实际毛利/采购成本）"
# 成本逾期率=逾期损失/采购金额

In [27]:

mon_zhys2 = mon_zhys2.rename(columns={ #'search_time':'下单月份'
                                        'order_number':'订单数'
                                        ,'purchase_amount':'采购金额'
                                        ,'all_deposit':'总押金'
                                        ,'all_rental':'总租金'
                                        ,'buyout_balance_payment':'预估残值'
                                        ,'total_receivable2':'总应收'
                                        ,'rent_received':'已收租金'
                                        ,'unearned_rent_revenue':'未收租金'
                                        ,'paid_buyout_balance_payment':'已归还残值/已付买断尾款'
                                        ,'total_received':'总已收'
                                        ,'total_unreceived':'总未收'
                                        ,'unrecovered_cost':'未回收成本'
                                        ,'overdue_total_unreceived':'逾期总未收'
                                        ,'overdue_rate':'应收逾期率'
                                        ,'overdue_loss':'逾期损失'
                                        ,'loss_rate':'成本逾期率'
                                        ,'gross_profit_receivable2':'应收毛利'
                                        ,'receivable_profite_rate':'应收毛利率(应收毛利/采购成本)'
                                        ,'actual_gross_profit':'实际毛利'
                                        ,'profite_rate':'实际毛利率 (实际毛利/采购成本)'})

In [28]:
mon_zhys2['预计坏账(逾期总未收/采购金额)'] = mon_zhys2['逾期总未收']/ mon_zhys2['采购金额']
mon_zhys2['实际毛利(应收毛利-预计坏账)'] = mon_zhys2['应收毛利率(应收毛利/采购成本)'] - mon_zhys2['预计坏账(逾期总未收/采购金额)']
mon_zhys2.head()

,下单日期,订单数,采购金额,采购金额2,总押金,总租金,预估残值,预估残值2,总应收,已收租金,...,应收逾期率,逾期损失,成本逾期率,应收毛利,应收毛利率(应收毛利/采购成本),实际毛利,实际毛利2,实际毛利率 (实际毛利/采购成本),预计坏账(逾期总未收/采购金额),实际毛利(应收毛利-预计坏账)
0,2022-07-05,1,5150.0,5150.0,5303.0,4295.40,3181.80,2188.75,6484.15,7847.20,...,0.000000,0.00,0.000000,1334.15,0.259058,3026.92,3026.92,0.587751,0.000000,0.259058
1,2022-07-15,1,4794.0,4794.0,5999.0,4799.16,2549.57,2037.45,6836.61,4799.16,...,0.000000,0.00,0.000000,2042.61,0.426076,2604.07,2604.07,0.543194,0.000000,0.426076
2,2022-07-16,1,9180.0,9180.0,9799.0,6859.20,5879.40,3901.50,10760.70,6859.20,...,0.000000,0.00,0.000000,1580.70,0.172190,3966.80,3966.80,0.432113,0.000000,0.172190
3,2022-07-17,2,4692.0,4692.0,6098.0,4929.69,2591.65,1994.10,6923.79,4929.69,...,0.000000,0.00,0.000000,2231.79,0.475659,3235.00,3235.00,0.689471,0.000000,0.475659
4,2022-07-18,2,6678.0,6678.0,7749.0,6232.80,3879.58,2838.15,9070.95,4699.80,...,0.375106,1491.76,0.223384,2392.95,0.358333,65.24,1543.39,0.009769,0.509519,-0.151186


In [29]:
# mon_zhys2_all = mon_zhys2.sum(axis=1)
mon_zhys2_all = mon_zhys2
mon_zhys2_all

,下单日期,订单数,采购金额,采购金额2,总押金,总租金,预估残值,预估残值2,总应收,已收租金,...,应收逾期率,逾期损失,成本逾期率,应收毛利,应收毛利率(应收毛利/采购成本),实际毛利,实际毛利2,实际毛利率 (实际毛利/采购成本),预计坏账(逾期总未收/采购金额),实际毛利(应收毛利-预计坏账)
0,2022-07-05,1,5150.0,5150.0,5303.0,4295.40,3181.80,2188.750,6484.150,7847.20,...,0.000000,0.00,0.000000,1334.150,0.259058,3026.92,3026.920,0.587751,0.000000,0.259058
1,2022-07-15,1,4794.0,4794.0,5999.0,4799.16,2549.57,2037.450,6836.610,4799.16,...,0.000000,0.00,0.000000,2042.610,0.426076,2604.07,2604.070,0.543194,0.000000,0.426076
2,2022-07-16,1,9180.0,9180.0,9799.0,6859.20,5879.40,3901.500,10760.700,6859.20,...,0.000000,0.00,0.000000,1580.700,0.172190,3966.80,3966.800,0.432113,0.000000,0.172190
3,2022-07-17,2,4692.0,4692.0,6098.0,4929.69,2591.65,1994.100,6923.790,4929.69,...,0.000000,0.00,0.000000,2231.790,0.475659,3235.00,3235.000,0.689471,0.000000,0.475659
4,2022-07-18,2,6678.0,6678.0,7749.0,6232.80,3879.58,2838.150,9070.950,4699.80,...,0.375106,1491.76,0.223384,2392.950,0.358333,65.24,1543.390,0.009769,0.509519,-0.151186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,2024-11-20,108,892667.0,923665.0,1017059.5,774137.36,544704.16,390423.375,1164560.735,97358.36,...,0.000000,0.00,0.000000,240895.735,0.269861,426174.56,240895.735,0.477417,0.000000,0.269861
831,2024-11-21,89,775984.0,761003.0,842521.6,618464.60,448442.40,321181.650,939646.250,72055.19,...,0.000000,0.00,0.000000,178643.250,0.230215,290922.99,178643.250,0.374908,0.000000,0.230215
832,2024-11-22,105,928122.0,935698.0,1018192.7,744686.98,582192.27,395949.900,1140636.880,88741.12,...,0.000000,0.00,0.000000,204938.880,0.220810,398757.26,204938.880,0.429639,0.000000,0.220810
833,2024-11-23,70,548783.0,613911.0,674461.4,480697.78,374295.38,259405.550,740103.330,54921.94,...,0.000000,0.00,0.000000,126192.330,0.229949,306210.15,126192.330,0.557980,0.000000,0.229949


In [30]:
Today1 = datetime.now().strftime('%Y%m%d')
mon_zhys2.to_excel(f'F:/myfile/业务相关数据/租后统计-{Today1}_new_4.xlsx')

In [31]:
df_zhys["月份"]=df_zhys["order_create_time"].dt.month
df_zhys["年份"]=df_zhys["order_create_time"].dt.year
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
df_zhys["季度"]= df_zhys.apply(lambda x:getjidu(x["月份"]),axis=1) 

df_zhys.groupby(['年份','季度']).agg({'order_id':'size','total_receivable':'sum'})

order_id  total_receivable
年份   季度                             
2022 3季度       122        1112667.88
     4季度       151        1554172.32
2023 1季度       867        8153935.31
     2季度      1701       16270124.41
     3季度      2626       26584638.34
     4季度      3726       37316976.33
2024 1季度      1910       18175198.06
     2季度      4470       41317232.17
     3季度      6872       62958209.02
     4季度      3457       40001868.53

# 剔除商家数据

In [32]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)


    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(25631, 76)

# 历史订单渠道名称补充

In [33]:

qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

In [34]:
# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

In [35]:
# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

np.int64(0)

# 渠道归属

In [36]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a


In [37]:
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"]),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

# 判断颜色 内存

In [38]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
dfzh1["颜色"].value_counts()[1:10]

颜色
原色钛金属         4547
黑色钛金属         3201
暗紫色           1818
深空黑色          1366
蓝色钛金属         1305
银色             873
黑色             868
蓝色钛金属-现货速发     812
蓝色             569
Name: count, dtype: int64

# 判断二手

In [39]:
dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    25552
二手       79
Name: count, dtype: int64

# 日期处理

In [40]:

dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

In [41]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
3季度    9471
4季度    7299
2季度    6086
1季度    2775
Name: count, dtype: int64

# 买断数据获取

In [42]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
SELECT t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(2433, 7)

In [43]:
df_out.columns

Index(['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay',
       'order_finish_date', 'finish_date_new'],
      dtype='object')

# 租后数据 关联 买断数据

In [44]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [45]:
# dfzh2[['order_finish_date','订单完成时间','pay_date','status']].info()

In [46]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(3044)

In [47]:

dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()


if_outpay
未知      23286
已买断      2298
部分买断       54
Name: count, dtype: int64

# 分期 取消数据 获取

In [48]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(1463, 7)

In [49]:
# df_cancel.columns

# 租后数据 关联 取消数据

In [50]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [51]:
# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

In [52]:
# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

In [53]:
# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})


In [54]:
# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})


# 按风控策略调整时间节点划分时间段

In [55]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

# 账期数据提取   含续租

## 获取续租分期数据

In [56]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(343777, 23)

## 重命名

In [57]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期'
,'reality_refund_date':'实付日期','sort':'当前期数'})

In [58]:
# df_ddfq = df_ddfq.rename(columns={'排序还款期数':'当前期数'})
# df_ddfq.columns

In [59]:
# df_xzfq.columns

## 实还金额核查

In [60]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

In [61]:
# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

In [62]:
# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())

# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

In [63]:
# total_check2.to_excel('F:/myfile/业务相关数据/total_check_0709-1.xlsx')

# 数据合并

In [64]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')

df_xz_concat['当前期数'].notnull().sum()

np.int64(338406)

In [65]:
df_xz_concat['order_id'].nunique()

25631

In [66]:
dfzh3['order_id'].nunique()

25631

In [67]:
df_xz_concat[["order_id","当前期数"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338451 entries, 0 to 338450
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   order_id  338451 non-null  int64  
 1   当前期数      338406 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 5.2 MB


# 去重

In [68]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('F:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

       order_id  当前期数
8039     119321   1.0
8051     119321   1.0
8040     119321   2.0
8052     119321   2.0
8041     119321   3.0
...         ...   ...
64982    552441  10.0
64971    552441  11.0
64983    552441  11.0
64972    552441  12.0
64984    552441  12.0

[168 rows x 2 columns]


In [69]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)

In [70]:
# df_xz_concat.to_excel('F:/myfile/业务相关数据/df_xz_concat.xlsx')

## 异常查询

In [71]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       '归属渠道', '是否二手', 'outpay_money', 'pay_date', 'status_y', 'if_outpay',
       'finish_date_new', 'nn', 'reality_refund_date', 'real_pay_money'],
      dtype='object', length=110)

In [72]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间

In [73]:
df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

,order_number,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new
2752,A202211111913512,650.70,79.00,2023-09-11,NaT,1.0,11.0,已逾期,NaT,5,NaT
4856,A202301071415408,915.00,200.00,2024-05-11,NaT,1.0,17.0,已逾期,NaT,4,NaT
10560,A2023022108454753,641.86,246.42,2023-09-24,NaT,1.0,8.0,还款中,NaT,5,NaT
14882,A2023032515133228,1499.22,565.00,2024-07-27,NaT,1.0,17.0,已逾期,NaT,4,NaT
15535,A2023033003351727,1084.73,700.00,2024-07-01,NaT,1.0,16.0,已逾期,NaT,4,NaT
25074,A2023050408232337,670.85,412.63,2024-12-06,NaT,1.0,20.0,续租中,NaT,4,NaT
40529,A20230627115744111,479.73,201.35,2024-03-30,NaT,1.0,10.0,还款中,NaT,5,NaT
41874,A2023070115535442,446.07,53.93,2024-05-04,NaT,1.0,11.0,还款中,NaT,5,NaT
60724,A2023081019051714,1074.81,700.00,2024-07-13,NaT,1.0,12.0,已买断,NaT,8,NaT
113803,A20231112005735116,732.98,368.38,2025-01-14,NaT,1.0,15.0,续租中,NaT,4,NaT


part_payment==0  reality_refund_date.notnull()  还款状态== '续租中' part_payment=money   还款状态== '已逾期' reality_refund_date=pd.na

In [74]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
90,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18 19:54:59,2.0,7.0,已逾期,NaT,5,NaT,NaN
91,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18 19:55:00,2.0,8.0,已逾期,NaT,5,NaT,NaN
92,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18 19:55:01,2.0,9.0,已逾期,NaT,5,NaT,NaN
125,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18 19:55:02,2.0,6.0,已逾期,NaT,4,NaT,NaN
126,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18 19:55:03,2.0,7.0,已逾期,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338078,A2024112401292337,1763177,0.00,0.0,2024-11-27,2024-11-24 01:29:53,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
338090,A2024112402270134,1763253,0.00,0.0,2024-11-27,2024-11-24 09:42:19,3.0,1.0,还款中,NaT,3,NaT,NaN
338163,A20241124111603118,1764176,0.00,0.0,2024-11-27,2024-11-24 11:37:29,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
338259,A20241124144249228,1765183,0.00,0.0,2024-11-27,2024-11-24 14:43:53,3.0,1.0,未到首期还款日,NaT,2,NaT,NaN


In [75]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
90,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18 19:54:59,2.0,7.0,已逾期,NaT,5,NaT,NaN
91,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18 19:55:00,2.0,8.0,已逾期,NaT,5,NaT,NaN
92,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18 19:55:01,2.0,9.0,已逾期,NaT,5,NaT,NaN
125,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18 19:55:02,2.0,6.0,已逾期,NaT,4,NaT,NaN
126,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18 19:55:03,2.0,7.0,已逾期,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338078,A2024112401292337,1763177,0.00,0.0,2024-11-27,2024-11-24 01:29:53,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
338090,A2024112402270134,1763253,0.00,0.0,2024-11-27,2024-11-24 09:42:19,3.0,1.0,还款中,NaT,3,NaT,NaN
338163,A20241124111603118,1764176,0.00,0.0,2024-11-27,2024-11-24 11:37:29,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
338259,A20241124144249228,1765183,0.00,0.0,2024-11-27,2024-11-24 14:43:53,3.0,1.0,未到首期还款日,NaT,2,NaT,NaN


实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空

In [76]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test.订单状态值.value_counts()

订单状态值
5    174
4    101
3      5
2      2
Name: count, dtype: int64

实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消

In [77]:
dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test.订单状态值.value_counts()

订单状态值
10    1416
8       46
4       14
5       11
Name: count, dtype: int64

部分还款，实付日期非空，完成时间为空 用户：  实付日期清空  

In [78]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test1

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
1956,A202210092154098,27461,591.60,408.40,2023-01-09 00:00:00,2023-03-18 20:04:40,2.0,4.0,已逾期,NaT,5,NaT,NaN
49029,A2023071909113511,477555,583.26,200.00,2024-02-22 00:00:00,2024-02-28 20:07:18,1.0,8.0,已逾期,NaT,5,NaT,NaN
125611,A2023112606505333,814549,541.32,22.76,2024-08-29 00:00:00,2024-08-12 17:47:02,1.0,10.0,已逾期,NaT,4,NaT,NaN
282608,A20240915113026156,1475789,544.92,534.02,2024-11-18 00:00:00,2024-11-14 00:37:19,3.0,3.0,还款中,NaT,4,NaT,NaN
285836,A20240914145305347,1471829,499.85,474.85,2024-11-23 22:25:25,2024-11-23 21:08:24,3.0,3.0,还款中,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338029,A20241123194943105,1762651,527.12,509.45,2025-10-26 00:00:00,2024-11-23 20:01:11,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN
338091,A2024112402270134,1763253,487.73,464.17,2024-12-27 00:00:00,2024-11-24 09:49:33,3.0,2.0,还款中,NaT,3,NaT,NaN
338186,A20241124125617266,1764662,383.34,383.33,2025-10-27 00:00:00,2024-11-24 15:08:41,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN
338270,A20241124144249228,1765183,512.38,488.81,2025-10-27 00:00:00,2024-11-24 14:46:06,3.0,12.0,未到首期还款日,NaT,2,NaT,NaN


 部分还款，实付日期为空，完成时间非空 用户：  实付日期=完成时间

In [79]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test1.订单状态值.value_counts()

订单状态值
8    157
Name: count, dtype: int64

已买断 但无实付日期及金额：实付日回写

In [80]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

# dd_test1.订单状态值.value_counts()
dd_test1

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money
22,A202207050851373,8924,354.00,0.0,2024-05-01,NaT,1.0,23.0,已买断,2024-04-17 01:37:33,8,2024-04-17 01:37:33,NaN,329.72
23,A202207050851373,8924,47.20,0.0,2024-05-31,NaT,1.0,24.0,已买断,2024-04-17 01:37:33,8,2024-04-17 01:37:33,NaN,329.72
407,A202208032116483,12219,483.00,0.0,2023-12-31,NaT,1.0,18.0,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
408,A202208032116483,12219,483.00,0.0,2024-01-30,NaT,1.0,19.0,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
409,A202208032116483,12219,483.00,0.0,2024-02-29,NaT,1.0,20.0,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172015,A2024041316125534,995895,1061.16,0.0,2024-11-16,NaT,1.0,8.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
172016,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
172017,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
172018,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38


##  实还日期 修正

In [81]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [82]:
# 部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)
df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']]                            

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
1956,A202210092154098,27461,591.60,408.40,2023-01-09 00:00:00,2023-03-18,2.0,4.0,已逾期,NaT,5,NaT,NaN,NaT
49029,A2023071909113511,477555,583.26,200.00,2024-02-22 00:00:00,2024-02-28,1.0,8.0,已逾期,NaT,5,NaT,NaN,NaT
125611,A2023112606505333,814549,541.32,22.76,2024-08-29 00:00:00,2024-08-12,1.0,10.0,已逾期,NaT,4,NaT,NaN,NaT
282608,A20240915113026156,1475789,544.92,534.02,2024-11-18 00:00:00,2024-11-14,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
285836,A20240914145305347,1471829,499.85,474.85,2024-11-23 22:25:25,2024-11-23,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338029,A20241123194943105,1762651,527.12,509.45,2025-10-26 00:00:00,2024-11-23,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
338091,A2024112402270134,1763253,487.73,464.17,2024-12-27 00:00:00,2024-11-24,3.0,2.0,还款中,NaT,3,NaT,NaN,NaT
338186,A20241124125617266,1764662,383.34,383.33,2025-10-27 00:00:00,2024-11-24,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
338270,A20241124144249228,1765183,512.38,488.81,2025-10-27 00:00:00,2024-11-24,3.0,12.0,未到首期还款日,NaT,2,NaT,NaN,NaT


In [83]:
# 已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']]    

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
1956,A202210092154098,27461,591.60,408.40,2023-01-09,2023-03-18,2.0,4.0,已逾期,NaT,5,NaT,NaN,NaT
2752,A202211111913512,35537,650.70,79.00,2023-09-11,NaT,1.0,11.0,已逾期,NaT,5,NaT,NaN,NaT
4856,A202301071415408,87888,915.00,200.00,2024-05-11,NaT,1.0,17.0,已逾期,NaT,4,NaT,NaN,NaT
10560,A2023022108454753,144517,641.86,246.42,2023-09-24,NaT,1.0,8.0,还款中,NaT,5,NaT,9.0,NaT
14882,A2023032515133228,193214,1499.22,565.00,2024-07-27,NaT,1.0,17.0,已逾期,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338029,A20241123194943105,1762651,527.12,509.45,2025-10-26,2024-11-23,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
338091,A2024112402270134,1763253,487.73,464.17,2024-12-27,2024-11-24,3.0,2.0,还款中,NaT,3,NaT,NaN,NaT
338186,A20241124125617266,1764662,383.34,383.33,2025-10-27,2024-11-24,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
338270,A20241124144249228,1765183,512.38,488.81,2025-10-27,2024-11-24,3.0,12.0,未到首期还款日,NaT,2,NaT,NaN,NaT


In [84]:
# 租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60724,2024-07-22


In [85]:
# 未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] 


,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
90,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18,2.0,7.0,已逾期,NaT,5,NaT,NaN,NaT
91,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18,2.0,8.0,已逾期,NaT,5,NaT,NaN,NaT
92,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18,2.0,9.0,已逾期,NaT,5,NaT,NaN,NaT
125,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18,2.0,6.0,已逾期,NaT,4,NaT,NaN,NaT
126,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18,2.0,7.0,已逾期,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338078,A2024112401292337,1763177,0.00,0.0,2024-11-27,2024-11-24,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN,NaT
338090,A2024112402270134,1763253,0.00,0.0,2024-11-27,2024-11-24,3.0,1.0,还款中,NaT,3,NaT,NaN,NaT
338163,A20241124111603118,1764176,0.00,0.0,2024-11-27,2024-11-24,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN,NaT
338259,A20241124144249228,1765183,0.00,0.0,2024-11-27,2024-11-24,3.0,1.0,未到首期还款日,NaT,2,NaT,NaN,NaT


In [86]:
# 实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] 

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
22,A202207050851373,8924,354.00,0.0,2024-05-01,NaT,1.0,23.0,已买断,2024-04-17 01:37:33,8,2024-04-17,NaN,2024-04-17
23,A202207050851373,8924,47.20,0.0,2024-05-31,NaT,1.0,24.0,已买断,2024-04-17 01:37:33,8,2024-04-17,NaN,2024-04-17
407,A202208032116483,12219,483.00,0.0,2023-12-31,NaT,1.0,18.0,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
408,A202208032116483,12219,483.00,0.0,2024-01-30,NaT,1.0,19.0,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
409,A202208032116483,12219,483.00,0.0,2024-02-29,NaT,1.0,20.0,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172015,A2024041316125534,995895,1061.16,0.0,2024-11-16,NaT,1.0,8.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
172016,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
172017,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
172018,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17


In [87]:
# 部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [88]:
# 未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [89]:
# 实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [90]:
# 属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [91]:
# df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [92]:
# A202207050851373 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number=='A202207050851373'),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money
321738,A20241107115806334,1693433,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
322255,A20241107114721284,1693383,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323096,A2024110815063348,1698235,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323097,A2024110815083268,1698255,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323266,A20241109103005181,1700566,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323267,A20241109103342198,1700583,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323280,A20241109105443311,1700696,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323305,A2024110911115785,1700810,NaN,NaN,None,NaT,NaN,NaN,还款中,NaT,4,NaT,NaN,NaN
323318,A20241109111655116,1700841,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
323355,A20241109115718348,1701073,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN


In [93]:
# 实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)



In [94]:
# df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 获取逾期截止时间

In [95]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date']

0        2024-11-25
1        2024-11-25
2        2024-11-25
3        2024-11-25
4        2024-11-25
            ...    
338446   2024-11-25
338447   2024-11-25
338448   2024-11-25
338449   2024-11-25
338450   2024-11-25
Name: now_date, Length: 338367, dtype: datetime64[s]

## 重新定义状态

In [96]:
# df_xz_concat.columns

In [97]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()

rembursement_status
还款中                    69730
已逾期                    32725
未到首期还款日                20892
续租中                    17255
检测中                       24
Name: count, dtype: int64

In [98]:
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

订单状态
租赁中     132902
待归还       6214
已退款       1416
待收货       1300
待发货        186
已完成         56
检测中         24
Name: count, dtype: int64

In [99]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     150855
未到还款日    141176
已逾期支付     24061
已逾期       20758
已取消        1472
其他           45
Name: count, dtype: int64

In [100]:
df_xz_concat.head()

,id_x,order_id,当前应付租金,实付金额,interest_money,应付日期,实付日期,status_x,当前期数,order_pay_id,...,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态
0,6506.0,8924,357.95,357.95,0.0,1656979200000000000,2022-07-05,3.0,1.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2024-11-25,正常还款
1,6507.0,8924,357.95,357.95,0.0,1659657600000000000,2022-08-05,3.0,2.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-08-05,2022-08-05,2024-11-25,正常还款
2,6508.0,8924,357.95,357.95,0.0,1662336000000000000,2022-09-05,3.0,3.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-09-05,2022-09-05,2024-11-25,正常还款
3,6509.0,8924,357.95,357.95,0.0,1664928000000000000,2022-10-09,3.0,4.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-10-09,2022-10-09,2024-11-25,已逾期支付
4,6510.0,8924,357.95,357.95,0.0,1667606400000000000,2022-11-08,3.0,5.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-11-08,2022-11-08,2024-11-25,已逾期支付


In [101]:
df_xz_concat.shape

(338367, 114)

In [102]:
df_xz_concat.实付日期new.value_counts()

实付日期new
2024-10-15    889
2024-11-15    871
2024-11-20    771
2024-10-18    750
2024-11-04    748
             ... 
2022-10-24      1
2022-07-21      1
2022-09-24      1
2022-09-27      1
2022-07-05      1
Name: count, Length: 864, dtype: int64

In [103]:
df_xz_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 338367 entries, 0 to 338450
Columns: 114 entries, id_x to 状态
dtypes: datetime64[ns](12), datetime64[s](1), float64(50), int32(2), int64(14), object(35)
memory usage: 294.3+ MB


In [104]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

In [105]:
# # F:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('F:/myfile/业务相关数据/报错表0902.xlsx')

## 重新定义逾期天数

In [106]:
df_xz_concat['实付日期new'].isnull().sum()

np.int64(142098)

In [107]:
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

321738    A20241107115806334
322255    A20241107114721284
323096     A2024110815063348
323097     A2024110815083268
323266    A20241109103005181
323267    A20241109103342198
323280    A20241109105443311
323305     A2024110911115785
323318    A20241109111655116
323355    A20241109115718348
323848    A20241109105400309
324221     A2024111012023021
325662    A20241111151909143
325747    A20241111155614415
325916      A202411111700326
326373    A20241112103431234
327394    A20241112103215217
327395     A2024111211052040
327396    A20241112113839253
327397    A20241112144927259
328694    A20241114143539203
329415    A20241115122728178
329416    A20241115124143262
329945    A20241115121741112
329958      A202411151401213
329959    A20241115142433138
330320     A2024111614051927
330693     A2024111616060624
330694    A20241116165435249
331619     A2024111812084173
331764    A20241118134701142
333049    A20241119135555368
333074    A20241119135915393
333135    A20241119141654130
333616      A2

In [108]:
empty_rows.order_number 

321738    A20241107115806334
322255    A20241107114721284
323096     A2024110815063348
323097     A2024110815083268
323266    A20241109103005181
323267    A20241109103342198
323280    A20241109105443311
323305     A2024110911115785
323318    A20241109111655116
323355    A20241109115718348
323848    A20241109105400309
324221     A2024111012023021
325662    A20241111151909143
325747    A20241111155614415
325916      A202411111700326
326373    A20241112103431234
327394    A20241112103215217
327395     A2024111211052040
327396    A20241112113839253
327397    A20241112144927259
328694    A20241114143539203
329415    A20241115122728178
329416    A20241115124143262
329945    A20241115121741112
329958      A202411151401213
329959    A20241115142433138
330320     A2024111614051927
330693     A2024111616060624
330694    A20241116165435249
331619     A2024111812084173
331764    A20241118134701142
333049    A20241119135555368
333074    A20241119135915393
333135    A20241119141654130
333616      A2

In [109]:
pd.to_datetime(df_xz_concat['实付日期new'])
# pd.to_datetime(df_xz_concat['应付日期'])

0        2022-07-05
1        2022-08-05
2        2022-09-05
3        2022-10-09
4        2022-11-08
            ...    
338446          NaT
338447          NaT
338448          NaT
338449          NaT
338450   2024-11-24
Name: 实付日期new, Length: 338367, dtype: datetime64[ns]

In [110]:
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(338322, 114)

In [111]:

# # 实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

df_xz_concat['逾期天数'].value_counts()

逾期天数
0      293516
1        5246
2        2865
3        2327
4        2052
        ...  
828         1
678         1
709         1
770         1
647         1
Name: count, Length: 730, dtype: int64

In [112]:
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        30859
还款中         5073
续租中         4193
已买断         3843
已完成          674
检测中          125
未到首期还款日       39
Name: count, dtype: int64

# 逾期天数处理

In [113]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

##  核查 

In [114]:
# df_ddfq.columns

In [115]:
# df_xz_concat.shape

In [116]:
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape

In [117]:
# check_fq_df.columns

In [118]:
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('F:/myfile/业务相关数据/check_fq_df.xlsx')

# 确认是否到表现期

In [119]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [120]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [121]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 


In [122]:

from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','应付日期']].head(10)

,agr_30d,应付日期
0,2022-08-04,2022-07-05
1,2022-09-04,2022-08-05
2,2022-10-05,2022-09-05
3,2022-11-04,2022-10-05
4,2022-12-05,2022-11-05
5,2023-01-04,2022-12-05
6,2023-02-04,2023-01-05
7,2023-03-07,2023-02-05
8,2023-04-04,2023-03-05
9,2023-05-05,2023-04-05


In [123]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(10)

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2024-11-25,1
1,2022-09-04,2022-08-05,2024-11-25,1
2,2022-10-05,2022-09-05,2024-11-25,1
3,2022-11-04,2022-10-05,2024-11-25,1
4,2022-12-05,2022-11-05,2024-11-25,1
5,2023-01-04,2022-12-05,2024-11-25,1
6,2023-02-04,2023-01-05,2024-11-25,1
7,2023-03-07,2023-02-05,2024-11-25,1
8,2023-04-04,2023-03-05,2024-11-25,1
9,2023-05-05,2023-04-05,2024-11-25,1


# 观察日定义  每月月底时间

In [124]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(0,13):  
        df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)
df_xz_concat[['下单日期','mob_date_0','mob_date_1','mob_date_2','mob_date_3','实付金额','paid_money_mob1','paid_date_mob12','paid_money_mob12']]

,下单日期,mob_date_0,mob_date_1,mob_date_2,mob_date_3,实付金额,paid_money_mob1,paid_date_mob12,paid_money_mob12
0,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,357.95,2022-07-05,357.95
1,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,357.95,2022-08-05,357.95
2,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-09-05,357.95
3,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-10-09,357.95
4,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-11-08,357.95
...,...,...,...,...,...,...,...,...,...
338446,2024-11-24,2024-11-30,2024-12-31,2025-01-31,2025-02-28,0.00,0.00,NaT,0.00
338447,2024-11-24,2024-11-30,2024-12-31,2025-01-31,2025-02-28,0.00,0.00,NaT,0.00
338448,2024-11-24,2024-11-30,2024-12-31,2025-01-31,2025-02-28,0.00,0.00,NaT,0.00
338449,2024-11-24,2024-11-30,2024-12-31,2025-01-31,2025-02-28,0.00,0.00,NaT,0.00


In [125]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [126]:
# df_xz_concat.columns

In [127]:
# df_xz_concat[df_xz_concat.order_id== 12514][['下单日期','mob_date_10','mob_date_11','实付金额','paid_money_mob10','paid_money_mob11','paid_date_mob12','paid_money_mob12']]


# 续租用户

In [128]:
df_xz_concat.is_relet.value_counts()

is_relet
0.0    307023
1.0     31238
2.0        61
Name: count, dtype: int64

In [129]:
df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
df_relet.shape

(31299, 165)

## 每个mob对应的逾期天数

In [130]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,24):  
        df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_relet = mob_date(df_relet)
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']]

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0
14,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,NaN,2023-09-06,354.0
15,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,365.8,NaN,2023-09-29,365.8
16,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,NaN,NaT,354.0
...,...,...,...,...,...,...,...,...,...
235873,2024-07-24,2025-10-31,2026-04-30,2026-03-31,2025-08-31,0.0,0.0,NaT,0.0
235874,2024-07-24,2025-10-31,2026-04-30,2026-03-31,2025-08-31,0.0,0.0,NaT,0.0
235875,2024-07-24,2025-10-31,2026-04-30,2026-03-31,2025-08-31,0.0,0.0,NaT,0.0
235876,2024-07-24,2025-10-31,2026-04-30,2026-03-31,2025-08-31,0.0,0.0,NaT,0.0


In [131]:
# 每个mob对应的逾期天数  mob24无逾期数据
df_mob_list = []
for i in range(11,24):  #mob24无逾期数据
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)]  
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)
df_mob_relet

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB
0,A202207050851373,2022-07-05,354.00,354.0,0,mob_12
1,A2022072220482114,2022-07-22,495.00,0.0,9,mob_12
2,A202207250005332,2022-07-25,293.40,293.4,0,mob_12
3,A2022072701455912,2022-07-27,297.00,0.0,3,mob_12
4,A202207282047042,2022-07-28,571.50,0.0,2,mob_12
...,...,...,...,...,...,...
8972,A202211232245095,2022-11-23,5403.28,5400.0,0,mob_23
8973,A202211241505071,2022-11-24,7267.50,7267.5,0,mob_23
8974,A2022112422562210,2022-11-24,5443.20,2016.0,220,mob_23
8975,A202211271518377,2022-11-27,7962.50,2205.0,281,mob_23


In [132]:
df_mob_relet['overdue_days_1'] = np.where(df_mob_relet['overdue_days_mob']>= 1,1,0)
df_mob_relet['overdue_days_4'] = np.where(df_mob_relet['overdue_days_mob']>= 4,1,0)
df_mob_relet['overdue_days_7'] = np.where(df_mob_relet['overdue_days_mob']>= 7,1,0)
df_mob_relet['overdue_days_15'] = np.where(df_mob_relet['overdue_days_mob']>= 15,1,0)
df_mob_relet['overdue_days_31'] = np.where(df_mob_relet['overdue_days_mob']>= 31,1,0)
df_mob_relet['下单月份'] = df_mob_relet['下单日期'].dt.strftime('%Y-%m')  

In [133]:
# df_mob_total['采购金额'] 
df_mob_relet = df_mob_relet.merge(dfzh1[['order_number','purchase_amount','归属渠道']],on = 'order_number',how='left')
df_mob_relet.head()

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,下单月份,purchase_amount,归属渠道
0,A202207050851373,2022-07-05,354.0,354.0,0,mob_12,0,0,0,0,0,2022-07,5150.0,未知渠道
1,A2022072220482114,2022-07-22,495.0,0.0,9,mob_12,1,1,1,0,0,2022-07,5890.0,其他渠道场景渠道。
2,A202207250005332,2022-07-25,293.4,293.4,0,mob_12,0,0,0,0,0,2022-07,3867.0,搜索渠道
3,A2022072701455912,2022-07-27,297.0,0.0,3,mob_12,1,0,0,0,0,2022-07,5280.0,其他渠道场景渠道。
4,A202207282047042,2022-07-28,571.5,0.0,2,mob_12,1,0,0,0,0,2022-07,9000.0,搜索渠道


In [134]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       'paid_money_mob9', 'mob_date_10', 'paid_date_mob10', 'paid_money_mob10',
       'mob_date_11', 'paid_date_mob11', 'paid_money_mob11', 'mob_date_12',
       'paid_date_mob12', 'paid_money_mob12'],
      dtype='object', length=165)

In [135]:
df_relet_mob10 = df_xz_concat[((df_xz_concat[f'mob_date_10'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_10']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob10':'sum'}).reset_index()

df_relet_mob10

,order_number,下单日期,当前应付租金,paid_money_mob10
0,A202207050851373,2022-07-05,3937.45,3937.45
1,A202207151707153,2022-07-15,4399.23,4399.23
2,A202207162356303,2022-07-16,6287.60,6287.60
3,A202207171820512,2022-07-17,1292.61,1175.10
4,A202207172051293,2022-07-17,3199.70,3199.70
...,...,...,...,...
9170,A2023123016303653,2023-12-30,5985.50,5985.50
9171,A202312301705428,2023-12-30,2948.68,2948.68
9172,A2023123017082113,2023-12-30,5330.20,5330.20
9173,A2023123017271634,2023-12-30,4739.41,2106.96


In [136]:
df_mob_relet = df_mob_relet.merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
df_mob_relet.head()

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,下单月份,purchase_amount,归属渠道,paid_money_mob10
0,A202207050851373,2022-07-05,354.0,354.0,0,mob_12,0,0,0,0,0,2022-07,5150.0,未知渠道,3937.45
1,A2022072220482114,2022-07-22,495.0,0.0,9,mob_12,1,1,1,0,0,2022-07,5890.0,其他渠道场景渠道。,3963.30
2,A202207250005332,2022-07-25,293.4,293.4,0,mob_12,0,0,0,0,0,2022-07,3867.0,搜索渠道,3225.97
3,A2022072701455912,2022-07-27,297.0,0.0,3,mob_12,1,0,0,0,0,2022-07,5280.0,其他渠道场景渠道。,3253.50
4,A202207282047042,2022-07-28,571.5,0.0,2,mob_12,1,0,0,0,0,2022-07,9000.0,搜索渠道,6261.60


## 续租订单 每月对应的续租用户及采购金额

In [137]:
df_relet.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       'paid_money_mob20', 'mob_date_21', 'paid_date_mob21',
       'paid_money_mob21', 'mob_date_22', 'paid_date_mob22',
       'paid_money_mob22', 'mob_date_23', 'paid_date_mob23',
       'paid_money_mob23'],
      dtype='object', length=198)

In [138]:
df_relet1 = df_relet[df_relet.当前期数==13].merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
# df_relet1
df_relet_mon = df_relet1.groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum','paid_money_mob10_y':'sum'})
df_relet_mon['relet_principal'] = df_relet_mon['purchase_amount'] - df_relet_mon['paid_money_mob10_y']
df_relet_mon

,order_number,实付金额,purchase_amount,paid_money_mob10_y,relet_principal
search_time,,,,,
2022-07,5,2010.90,29187.00,20641.82,8545.18
2022-08,15,5776.54,98008.00,65333.41,32674.59
2022-09,14,4931.40,92730.85,68930.70,23800.15
2022-10,15,5420.93,87371.00,64734.60,22636.40
2022-11,17,8727.60,132745.00,83282.80,49462.20
2022-12,21,10731.90,177135.00,116275.20,60859.80
2023-01,61,49187.69,391326.00,273352.00,117974.00
2023-02,72,67235.73,457244.00,321404.44,135839.56
2023-03,69,78635.43,472351.00,301927.04,170423.96


## 续租订单 vintage 4+ 

In [139]:
vintage_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,20.00%,40.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%
2022-08,15,13.33%,13.33%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%
2022-09,14,35.71%,14.29%,21.43%,28.57%,35.71%,42.86%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%
2022-11,17,nan%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%
2022-12,21,9.52%,9.52%,19.05%,19.05%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,4.92%,14.75%,22.95%,24.59%,27.87%,29.51%,27.87%,21.31%,21.31%,19.67%,nan%
2023-02,72,9.72%,18.06%,22.22%,27.78%,30.56%,36.11%,27.78%,25.00%,23.61%,nan%,nan%
2023-03,69,8.70%,13.04%,18.84%,15.94%,24.64%,28.99%,23.19%,23.19%,nan%,nan%,nan%
2023-04,137,10.95%,16.79%,21.17%,26.28%,29.93%,34.31%,29.93%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 4+ 

In [140]:
df_mob_relet['unpaid_money_mob'] = df_mob_relet['purchase_amount'] - df_mob_relet['paid_money_mob'] - df_mob_relet['paid_money_mob10']


In [141]:
# df_mob_relet.to_excel(f'F:/myfile/业务相关数据/df_mob_relet-{date_str}.xlsx')

In [142]:
money_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,22.55%,42.79%,42.79%,22.55%,22.55%,22.55%,16.56%,16.56%,-4.70%,-28.15%,nan%
2022-08,32674.59,7.94%,2.77%,14.46%,16.85%,15.73%,14.46%,14.69%,13.08%,13.08%,11.51%,11.51%
2022-09,23800.15,36.34%,16.46%,18.09%,16.58%,17.15%,17.45%,15.49%,11.18%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,4.91%,9.85%,8.82%,9.67%,6.41%,-3.23%,-2.33%,-0.07%,6.38%
2022-11,49462.20,nan%,9.76%,9.78%,13.42%,12.84%,9.55%,9.55%,9.55%,6.31%,9.55%,9.55%
2022-12,60859.80,13.13%,13.13%,19.95%,18.87%,14.36%,14.36%,13.31%,13.45%,12.13%,12.13%,12.13%
2023-01,117974.00,4.94%,8.75%,11.16%,9.09%,2.21%,-4.65%,-5.18%,-2.02%,-2.02%,-3.22%,nan%
2023-02,135839.56,9.28%,11.38%,9.87%,3.47%,-0.72%,-12.07%,-7.38%,-5.48%,-5.33%,nan%,nan%
2023-03,170423.96,9.25%,10.63%,10.33%,3.76%,-4.94%,-14.64%,-6.82%,-6.82%,nan%,nan%,nan%
2023-04,344818.84,11.75%,13.26%,12.00%,5.31%,-3.58%,-7.20%,-1.11%,nan%,nan%,nan%,nan%


## 续租订单 vintage 7+ 

In [143]:
vintage_relet_d7 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_7==1],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,20.00%,20.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%
2022-08,15,13.33%,6.67%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%
2022-09,14,7.14%,14.29%,14.29%,14.29%,28.57%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%
2022-11,17,nan%,nan%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%
2022-12,21,nan%,9.52%,14.29%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%
2023-01,61,4.92%,14.75%,21.31%,24.59%,27.87%,27.87%,27.87%,21.31%,21.31%,19.67%,nan%
2023-02,72,8.33%,16.67%,19.44%,25.00%,26.39%,33.33%,27.78%,25.00%,23.61%,nan%,nan%
2023-03,69,7.25%,11.59%,17.39%,14.49%,23.19%,28.99%,23.19%,23.19%,nan%,nan%,nan%
2023-04,137,10.22%,16.79%,20.44%,26.28%,29.93%,33.58%,29.93%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 7+ 

In [144]:
money_relet_d7 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_7==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,22.55%,22.55%,42.79%,22.55%,22.55%,22.55%,16.56%,16.56%,-4.70%,-28.15%,nan%
2022-08,32674.59,7.94%,0.14%,14.46%,16.85%,15.73%,14.46%,14.69%,13.08%,13.08%,11.51%,11.51%
2022-09,23800.15,7.24%,16.46%,16.46%,16.46%,16.58%,17.15%,15.49%,11.18%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,4.91%,9.85%,8.82%,9.67%,6.41%,-3.23%,-2.33%,-0.07%,6.38%
2022-11,49462.20,nan%,nan%,9.78%,13.42%,12.19%,9.55%,9.55%,9.55%,6.31%,9.55%,9.55%
2022-12,60859.80,nan%,13.13%,16.99%,18.87%,14.36%,14.36%,14.36%,13.45%,12.13%,12.13%,12.13%
2023-01,117974.00,4.94%,8.75%,10.70%,9.09%,2.21%,-4.07%,-5.18%,-2.02%,-2.02%,-3.22%,nan%
2023-02,135839.56,7.90%,10.61%,10.04%,5.40%,5.03%,-7.61%,-7.38%,-5.48%,-5.33%,nan%,nan%
2023-03,170423.96,7.08%,9.07%,10.33%,4.38%,-2.85%,-14.64%,-6.82%,-6.82%,nan%,nan%,nan%
2023-04,344818.84,10.97%,13.26%,11.96%,5.31%,-3.58%,-6.13%,-1.11%,nan%,nan%,nan%,nan%


## 续租订单 vintage 15+ 

In [145]:
vintage_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,nan%,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%
2022-08,15,6.67%,6.67%,20.00%,26.67%,26.67%,20.00%,26.67%,20.00%,26.67%,26.67%,26.67%
2022-09,14,nan%,7.14%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,nan%,20.00%,13.33%,33.33%,26.67%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,9.52%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%
2023-01,61,1.64%,11.48%,18.03%,24.59%,26.23%,24.59%,27.87%,21.31%,21.31%,19.67%,nan%
2023-02,72,5.56%,12.50%,19.44%,22.22%,25.00%,31.94%,27.78%,25.00%,23.61%,nan%,nan%
2023-03,69,4.35%,7.25%,13.04%,11.59%,18.84%,26.09%,23.19%,23.19%,nan%,nan%,nan%
2023-04,137,5.84%,14.60%,18.25%,18.98%,25.55%,29.93%,29.93%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 15+ 

In [146]:
money_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d15 = money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d15 = money_relet_d15.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,nan%,22.55%,42.79%,22.55%,22.55%,22.55%,16.75%,16.56%,16.75%,nan%,nan%
2022-08,32674.59,7.80%,0.14%,14.46%,16.85%,15.73%,14.46%,14.53%,14.46%,13.08%,11.51%,11.51%
2022-09,23800.15,nan%,7.02%,16.46%,16.46%,16.58%,17.15%,16.58%,13.66%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,nan%,9.85%,10.16%,9.67%,6.41%,-3.23%,3.03%,6.38%,6.38%
2022-11,49462.20,nan%,nan%,9.78%,9.78%,12.19%,7.45%,9.55%,9.55%,9.55%,9.55%,9.55%
2022-12,60859.80,nan%,13.13%,13.13%,18.87%,14.36%,14.36%,14.36%,13.45%,12.13%,12.13%,12.13%
2023-01,117974.00,2.02%,6.97%,10.45%,9.09%,3.39%,0.55%,-5.18%,-2.02%,-2.02%,-3.22%,nan%
2023-02,135839.56,5.09%,9.11%,10.04%,7.70%,5.72%,-4.70%,-7.38%,-5.48%,-5.33%,nan%,nan%
2023-03,170423.96,3.32%,6.24%,9.14%,5.70%,0.81%,-9.57%,-6.82%,-6.82%,nan%,nan%,nan%
2023-04,344818.84,7.09%,11.79%,11.91%,8.72%,1.20%,-3.18%,-1.11%,nan%,nan%,nan%,nan%


## 续租订单 vintage 31+ 

In [147]:
vintage_relet_d31 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_31==1],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23''mob_11',
# vintage_df_d4
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%
2022-08,15,6.67%,13.33%,20.00%,20.00%,20.00%,20.00%,20.00%,26.67%,26.67%,26.67%
2022-09,14,nan%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,6.67%,13.33%,26.67%,26.67%,20.00%,20.00%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,14.29%,9.52%,14.29%,14.29%,14.29%,14.29%,19.05%,19.05%
2023-01,61,1.64%,13.11%,19.67%,19.67%,21.31%,26.23%,21.31%,21.31%,19.67%,nan%
2023-02,72,8.33%,13.89%,19.44%,22.22%,30.56%,27.78%,25.00%,23.61%,nan%,nan%
2023-03,69,4.35%,11.59%,10.14%,14.49%,24.64%,23.19%,23.19%,nan%,nan%,nan%
2023-04,137,8.03%,13.14%,16.06%,21.90%,26.28%,29.93%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 31+ 

In [148]:
money_relet_d31 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_31==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23''mob_11',
# vintage_df_d4
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,22.55%,42.79%,22.55%,22.55%,22.55%,16.75%,16.56%,16.75%,nan%,nan%
2022-08,32674.59,0.14%,9.35%,14.46%,14.46%,14.46%,14.46%,14.46%,13.08%,11.51%,11.51%
2022-09,23800.15,nan%,16.46%,16.46%,16.58%,17.15%,16.58%,13.66%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,-0.31%,10.16%,8.82%,6.41%,5.92%,4.40%,6.38%,6.38%
2022-11,49462.20,nan%,9.76%,9.78%,9.78%,7.45%,9.55%,9.55%,9.55%,9.55%,9.55%
2022-12,60859.80,13.13%,13.13%,16.09%,13.13%,14.36%,14.36%,14.36%,14.36%,12.13%,12.13%
2023-01,117974.00,2.02%,8.33%,9.38%,7.79%,3.67%,-4.72%,-2.02%,-2.02%,-3.22%,nan%
2023-02,135839.56,8.31%,9.35%,8.41%,5.55%,-1.56%,-7.38%,-5.48%,-5.33%,nan%,nan%
2023-03,170423.96,3.20%,9.07%,6.27%,4.87%,-6.99%,-6.82%,-6.82%,nan%,nan%,nan%
2023-04,344818.84,8.16%,11.50%,10.26%,5.34%,1.19%,-1.11%,nan%,nan%,nan%,nan%


# 非续租租期  每个mob 观察日 对应的逾期天数

实付日期逻辑：如实付日期大于观察日，实付日期清空 


达到表现期逻辑：应付日期小于观察日，且应付日期小于当前日期 

逾期天数判断逻辑:如状态为已取消状态则0，如实付日期不为空则 0，如实付日期为空且观察日期大于当前日期则 当前日期-应付日期，否则 观察日期-应付日期

In [149]:
# 非续租租期数据
df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]

In [150]:
# df_fq_concat.columns

In [151]:
df_mob_list = []
for i in range(0,13):
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)
df_mob_total.tail(10)

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB
177569,A20231030165218150,2023-10-30,3598.52,3598.52,0,mob_12
177570,A202310301705079,2023-10-30,13996.60,2335.30,334,mob_12
177571,A2023103017062512,2023-10-30,5793.72,5793.72,0,mob_12
177572,A2023103017351364,2023-10-30,8048.20,8048.20,0,mob_12
177573,A2023103017440978,2023-10-30,7247.79,7247.79,0,mob_12
177574,A2023103019285033,2023-10-30,2044.00,2044.00,0,mob_12
177575,A2023103021450037,2023-10-30,7797.50,7797.50,0,mob_12
177576,A2023103021593647,2023-10-30,4345.36,4345.36,0,mob_12
177577,A2023103023335810,2023-10-30,3195.00,2396.43,182,mob_12
177578,A2023103023355911,2023-10-30,6447.99,6447.99,0,mob_12


In [152]:
# df_mob_total[df_mob_total.订单号=='A202208061505082']

In [153]:
df_mob_total['overdue_days_1'] = np.where(df_mob_total['overdue_days_mob']>= 1,1,0)
df_mob_total['overdue_days_4'] = np.where(df_mob_total['overdue_days_mob']>= 4,1,0)
df_mob_total['overdue_days_7'] = np.where(df_mob_total['overdue_days_mob']>= 7,1,0)
df_mob_total['overdue_days_15'] = np.where(df_mob_total['overdue_days_mob']>= 15,1,0)
df_mob_total['overdue_days_31'] = np.where(df_mob_total['overdue_days_mob']>= 31,1,0)
df_mob_total['下单月份'] = df_mob_total['下单日期'].dt.strftime('%Y-%m')  

In [154]:
# dfzh3.columns

In [155]:
# df_mob_total['采购金额'] 
df_mob_total = df_mob_total.merge(dfzh3[['order_number','purchase_amount','归属渠道']],on = 'order_number',how='left')
df_mob_total.shape

(177670, 14)

In [156]:
from datetime import datetime as dt    
today_date = dt.now().date()  
date_str = today_date.strftime('%Y-%m-%d')  

# df_mob_total.to_excel(f'F:/myfile/业务相关数据/df_mob_total-{date_str}.xlsx')

## 每月对应的 用户数及应还金额

### 总体

In [157]:
df_mon = df_fq_concat[df_fq_concat.当前期数==1].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,21,182954.00,179819.43,125405.00
2022-08,55,515908.04,506808.62,358451.00
2022-09,44,402654.00,400561.83,277982.75
2022-10,41,337968.00,342350.72,249543.09
2022-11,43,440057.00,442109.08,344258.81
2022-12,66,740353.20,755854.12,566276.59
2023-01,221,2173455.52,2136004.90,1533459.00
2023-02,332,3218625.44,3140251.37,2181552.82
2023-03,314,3044754.23,2877679.04,2079408.00


### 租物渠道

In [158]:
df_mon_zm = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='芝麻租物')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_zm

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-09,5,34573.00,32945.01,24742.68
2022-10,10,65644.00,61704.19,48835.00
2022-11,10,89430.00,83735.04,68730.22
2022-12,2,10106.20,9409.65,7657.00
2023-01,18,158715.70,149658.35,110070.00
2023-02,38,306451.62,250279.32,206619.00
2023-03,61,550399.00,467178.34,374912.00
2023-04,240,2442118.00,2102734.65,1628122.00
2023-05,144,1400800.00,1221674.55,905764.00


### 搜索渠道

In [159]:
df_mon_ss = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='搜索渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_ss

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,7,67151.00,67091.26,46581.00
2022-08,22,210198.00,204866.79,145114.00
2022-09,8,96962.00,97022.13,67126.34
2022-10,6,59404.00,63328.32,44442.63
2022-11,10,113610.00,115602.98,88153.59
2022-12,37,424727.00,437884.25,324164.59
2023-01,134,1337111.40,1314893.17,944943.00
2023-02,200,2015258.42,2007467.30,1369414.82
2023-03,150,1468754.23,1413934.32,1002057.00


### 抖音渠道

In [160]:
df_mon_dy = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_dy

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2024-04,3,21916.0,21921.15,15930.0
2024-05,3,17517.0,17877.71,12690.0
2024-06,49,306191.0,308035.90,220205.0
2024-07,106,605534.0,614302.96,443358.0
2024-08,53,392623.0,401634.44,271022.0
2024-09,8,41644.0,41855.22,30173.0
2024-11,2,14038.0,13355.48,9865.0


## vintage 4+ 订单 口径 

### 总体

In [161]:
vintage_df_d4 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_4==1],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4 = vintage_df_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d4
vintage_divide_d4 = vintage_df_d4.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4['订单量'] = df_mon["order_number"]
vintage_divide_d4[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,21,9.52%,23.81%,28.57%,33.33%,38.10%,38.10%,38.10%,38.10%,42.86%,42.86%,38.10%,38.10%
2022-08,55,14.55%,21.82%,27.27%,29.09%,30.91%,36.36%,38.18%,41.82%,41.82%,41.82%,41.82%,40.00%
2022-09,44,6.82%,11.36%,18.18%,25.00%,29.55%,27.27%,25.00%,27.27%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,21.95%,19.51%,17.07%,21.95%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,13.95%,13.95%,23.26%,23.26%,18.60%,20.93%,20.93%,18.60%,18.60%
2022-12,66,3.03%,7.58%,15.15%,19.70%,19.70%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,5.43%,11.31%,14.48%,16.29%,21.27%,25.34%,26.24%,28.51%,29.86%,31.22%,30.32%,29.41%
2023-02,332,8.13%,11.75%,15.96%,17.17%,20.48%,22.29%,23.49%,25.00%,24.40%,24.40%,23.49%,23.19%
2023-03,314,6.69%,12.42%,16.24%,20.06%,23.25%,26.43%,26.11%,27.71%,28.98%,30.89%,28.98%,28.34%
2023-04,633,3.63%,8.21%,13.11%,15.80%,18.33%,21.33%,22.27%,23.85%,25.12%,25.59%,25.91%,25.28%


### 芝麻租物

In [162]:
# 芝麻租物渠道
vintage_df_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4_zm = vintage_df_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_zm = vintage_df_d4_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d4_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,5.56%,11.11%,11.11%,11.11%,22.22%,27.78%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,2.63%,5.26%,7.89%,5.26%,5.26%,10.53%,7.89%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,4.92%,6.56%,11.48%,13.11%,14.75%,16.39%,18.03%,19.67%,18.03%,18.03%
2023-04,240,2.92%,6.25%,9.58%,10.83%,11.67%,16.67%,17.08%,18.33%,18.75%,19.58%,19.58%,19.58%
2023-05,144,nan%,4.17%,9.03%,9.03%,10.42%,9.03%,15.28%,18.06%,19.44%,18.06%,20.14%,18.75%
2023-06,261,2.68%,6.90%,11.11%,14.56%,16.09%,18.39%,21.46%,24.52%,24.52%,26.82%,26.05%,25.67%


### 搜索渠道

In [163]:
# 搜索渠道
vintage_df_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4_ss = vintage_df_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_ss = vintage_df_d4_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d4_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,57.14%,42.86%,42.86%,42.86%
2022-08,22,13.64%,18.18%,22.73%,22.73%,22.73%,31.82%,36.36%,45.45%,36.36%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,30.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,2.70%,10.81%,18.92%,21.62%,24.32%,24.32%,27.03%,32.43%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.97%,12.69%,16.42%,15.67%,20.15%,23.88%,25.37%,27.61%,27.61%,29.85%,28.36%,26.87%
2023-02,200,10.50%,15.00%,18.50%,20.50%,24.00%,27.00%,29.00%,30.00%,29.50%,29.50%,28.50%,28.50%
2023-03,150,10.67%,18.67%,24.67%,27.33%,30.00%,34.00%,32.67%,34.00%,34.67%,36.67%,34.67%,34.67%
2023-04,190,4.21%,11.05%,16.84%,18.95%,21.58%,21.58%,25.26%,26.84%,28.95%,28.95%,31.05%,29.47%


### 抖音渠道

In [164]:
# 抖音渠道
vintage_df_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_df_d4_dy = vintage_df_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d4_dy
vintage_df_d4_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))


MOB,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6
2024-04,nan%,nan%,33.33%,33.33%,33.33%,66.67%
2024-05,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,20.41%,26.53%,26.53%,38.78%,nan%,nan%
2024-07,16.04%,25.47%,34.91%,nan%,nan%,nan%
2024-08,20.75%,33.96%,nan%,nan%,nan%,nan%
2024-09,25.00%,nan%,nan%,nan%,nan%,nan%
2024-11,nan%,nan%,nan%,nan%,nan%,nan%


## vintage 7+ 订单 口径 

### 总体

In [165]:
vintage_df_d7 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_7==1],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7 = vintage_df_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d7
vintage_divide_d7 = vintage_df_d7.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7['订单量'] = df_mon["order_number"]
vintage_divide_d7[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,21,4.76%,23.81%,28.57%,33.33%,38.10%,38.10%,38.10%,38.10%,42.86%,42.86%,38.10%,38.10%
2022-08,55,10.91%,21.82%,27.27%,29.09%,30.91%,34.55%,38.18%,41.82%,41.82%,41.82%,41.82%,40.00%
2022-09,44,6.82%,11.36%,18.18%,20.45%,29.55%,27.27%,25.00%,25.00%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,21.95%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,11.63%,11.63%,20.93%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,1.52%,7.58%,13.64%,19.70%,18.18%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.98%,10.86%,14.48%,16.29%,19.46%,24.89%,24.43%,27.60%,29.86%,30.77%,30.32%,29.41%
2023-02,332,7.53%,10.84%,15.06%,16.87%,19.88%,21.39%,23.19%,24.70%,24.10%,24.10%,23.49%,23.19%
2023-03,314,4.78%,11.15%,14.65%,19.11%,22.29%,24.52%,24.84%,25.80%,28.98%,30.89%,28.98%,28.34%
2023-04,633,3.32%,7.42%,12.01%,15.17%,17.69%,20.85%,21.64%,23.38%,24.64%,25.28%,25.91%,25.28%


### 租物渠道

In [166]:
# 搜索渠道
vintage_df_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7_zm = vintage_df_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_zm = vintage_df_d7_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d7_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,nan%,2.63%,2.63%,2.63%,5.26%,7.89%,5.26%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,4.92%,6.56%,11.48%,11.48%,13.11%,16.39%,18.03%,19.67%,18.03%,18.03%
2023-04,240,2.92%,5.83%,8.75%,10.83%,11.67%,16.67%,16.67%,18.33%,18.75%,19.58%,19.58%,19.58%
2023-05,144,nan%,4.17%,8.33%,9.03%,9.03%,9.03%,15.28%,17.36%,19.44%,16.67%,19.44%,18.75%
2023-06,261,1.92%,5.75%,10.34%,13.03%,15.33%,17.62%,21.46%,22.99%,24.52%,26.44%,25.67%,25.67%


### 搜索渠道

In [167]:
# 搜索渠道
vintage_df_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7_ss = vintage_df_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_ss = vintage_df_d7_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d7_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,57.14%,42.86%,42.86%,42.86%
2022-08,22,13.64%,18.18%,22.73%,22.73%,22.73%,31.82%,36.36%,45.45%,36.36%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,30.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,2.70%,10.81%,18.92%,21.62%,24.32%,24.32%,27.03%,32.43%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.22%,12.69%,16.42%,15.67%,17.16%,23.13%,23.13%,26.12%,27.61%,29.10%,28.36%,26.87%
2023-02,200,10.00%,14.00%,18.00%,20.50%,23.00%,26.00%,29.00%,30.00%,29.50%,29.00%,28.50%,28.50%
2023-03,150,8.00%,17.33%,22.67%,26.00%,29.33%,32.67%,32.00%,32.00%,34.67%,36.67%,34.67%,34.67%
2023-04,190,3.16%,8.95%,15.79%,17.89%,20.00%,20.53%,23.68%,26.32%,28.95%,28.42%,31.05%,29.47%


## vintage 15+ 订单 口径 

### 总体

In [168]:
vintage_df_d15 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15 = vintage_df_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d15
vintage_divide_d15 = vintage_df_d15.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15['订单量'] = df_mon["order_number"]
vintage_divide_d15[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,21,nan%,19.05%,28.57%,28.57%,28.57%,33.33%,38.10%,38.10%,38.10%,38.10%,38.10%,38.10%
2022-08,55,5.45%,14.55%,20.00%,23.64%,29.09%,34.55%,38.18%,40.00%,40.00%,41.82%,41.82%,40.00%
2022-09,44,2.27%,9.09%,18.18%,18.18%,27.27%,27.27%,25.00%,25.00%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,19.51%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,4.65%,11.63%,9.30%,11.63%,11.63%,18.60%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,nan%,7.58%,9.09%,19.70%,16.67%,16.67%,22.73%,21.21%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.07%,9.95%,11.76%,15.84%,19.00%,24.43%,23.98%,26.70%,29.41%,28.96%,30.32%,29.41%
2023-02,332,4.82%,9.04%,13.86%,14.16%,18.98%,19.88%,21.39%,23.80%,24.10%,23.80%,23.49%,23.19%
2023-03,314,3.18%,8.92%,12.42%,16.56%,18.15%,22.93%,22.93%,23.89%,27.71%,29.94%,28.98%,28.34%
2023-04,633,2.21%,5.85%,10.11%,13.43%,15.64%,19.43%,20.54%,21.80%,23.70%,23.22%,25.43%,25.28%


### 租物渠道

In [169]:

vintage_df_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15_zm = vintage_df_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_zm = vintage_df_d15_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d15_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,nan%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,nan%,2.63%,2.63%,2.63%,5.26%,7.89%,5.26%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,nan%,1.64%,3.28%,4.92%,6.56%,11.48%,13.11%,14.75%,16.39%,19.67%,18.03%,18.03%
2023-04,240,2.08%,3.75%,7.08%,9.58%,10.83%,15.00%,15.83%,16.67%,18.75%,18.75%,19.58%,19.58%
2023-05,144,nan%,2.78%,8.33%,9.03%,9.03%,8.33%,15.28%,15.97%,17.36%,15.97%,19.44%,18.75%
2023-06,261,1.15%,5.36%,8.81%,12.26%,14.94%,16.86%,20.69%,22.22%,23.75%,25.29%,25.67%,25.67%


### 搜索渠道

In [170]:
# 搜索渠道
vintage_df_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15_ss = vintage_df_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_ss = vintage_df_d15_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d15_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,nan%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%
2022-08,22,9.09%,9.09%,13.64%,13.64%,22.73%,31.82%,36.36%,40.91%,36.36%,36.36%,36.36%,36.36%
2022-09,8,12.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,20.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,nan%,10.81%,13.51%,21.62%,21.62%,21.62%,24.32%,24.32%,29.73%,29.73%,29.73%,27.03%
2023-01,134,3.73%,11.19%,11.94%,15.67%,16.42%,22.39%,22.39%,24.63%,26.87%,26.12%,28.36%,26.87%
2023-02,200,5.50%,12.00%,17.00%,17.00%,22.50%,24.00%,27.00%,29.50%,29.50%,28.50%,28.50%,28.50%
2023-03,150,6.00%,13.33%,18.67%,24.00%,24.67%,29.33%,28.67%,29.33%,34.00%,36.00%,34.67%,34.67%
2023-04,190,2.11%,7.89%,12.11%,14.74%,16.32%,19.47%,22.11%,23.68%,26.84%,24.21%,29.47%,29.47%


## vintage 31+ 订单 口径 

### 总体

In [171]:
vintage_df_d31 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_31==1],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31 = vintage_df_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d31
vintage_divide_d31 = vintage_df_d31.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31['订单量'] = df_mon["order_number"]
vintage_divide_d31[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'mob_1',

MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,21,19.05%,28.57%,28.57%,28.57%,33.33%,38.10%,38.10%,38.10%,38.10%,38.10%,38.10%
2022-08,55,12.73%,18.18%,21.82%,27.27%,29.09%,32.73%,38.18%,36.36%,41.82%,40.00%,40.00%
2022-09,44,6.82%,11.36%,15.91%,25.00%,25.00%,25.00%,22.73%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,17.07%,12.20%,17.07%,17.07%,19.51%,19.51%,19.51%
2022-11,43,9.30%,9.30%,11.63%,11.63%,13.95%,16.28%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,4.55%,6.06%,16.67%,16.67%,16.67%,21.21%,21.21%,25.76%,25.76%,25.76%,24.24%
2023-01,221,5.43%,9.50%,12.22%,15.38%,20.81%,20.36%,23.53%,27.15%,27.60%,29.86%,29.41%
2023-02,332,6.63%,9.94%,12.95%,16.57%,17.17%,19.58%,21.99%,22.59%,23.19%,23.19%,23.19%
2023-03,314,6.69%,10.51%,14.97%,17.52%,20.70%,21.34%,23.25%,26.75%,28.34%,28.03%,28.34%
2023-04,633,3.16%,6.48%,10.27%,12.95%,15.80%,18.64%,19.75%,22.27%,22.12%,24.01%,25.28%


### 租物渠道

In [172]:

vintage_df_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31_zm = vintage_df_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d31_zm = vintage_df_d31_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d31_zm[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']] # ,'mob_1'


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,nan%,nan%,10.00%,10.00%,10.00%,nan%,10.00%,nan%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,2.63%,2.63%,2.63%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,3.28%,6.56%,9.84%,11.48%,14.75%,14.75%,18.03%,18.03%,18.03%
2023-04,240,2.50%,5.00%,7.92%,9.58%,11.67%,13.75%,15.42%,17.92%,17.50%,18.75%,19.58%
2023-05,144,nan%,3.47%,6.94%,7.64%,8.33%,11.11%,12.50%,15.97%,15.97%,17.36%,18.75%
2023-06,261,2.30%,6.13%,9.96%,12.26%,14.94%,17.62%,19.92%,22.22%,23.75%,25.29%,25.67%


### 搜索渠道

In [173]:
# 搜索渠道
vintage_df_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31_ss = vintage_df_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_d31_ss = vintage_df_d31_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d31_ss[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%
2022-08,22,9.09%,13.64%,13.64%,22.73%,22.73%,31.82%,36.36%,31.82%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,50.00%,50.00%,50.00%,62.50%,62.50%,62.50%,62.50%
2022-10,6,16.67%,16.67%,16.67%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,40.00%,30.00%,40.00%,40.00%,40.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,5.41%,8.11%,18.92%,21.62%,21.62%,24.32%,24.32%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.22%,8.96%,12.69%,13.43%,18.66%,19.40%,22.39%,24.63%,24.63%,27.61%,26.87%
2023-02,200,8.50%,12.00%,15.50%,20.00%,20.50%,24.50%,27.00%,27.50%,28.00%,28.00%,28.50%
2023-03,150,10.67%,16.67%,23.33%,24.00%,27.33%,26.67%,28.67%,33.33%,34.67%,33.33%,34.67%
2023-04,190,3.68%,7.37%,12.63%,15.26%,16.32%,19.47%,21.05%,25.26%,24.21%,27.37%,29.47%


## 金额 vintage 4+  口径 

In [174]:
# df_mob_total.columns

In [175]:
df_mob_total['unpaid_money_mob'] = df_mob_total['purchase_amount'] - df_mob_total['paid_money_mob']

### 总体

In [176]:
vintage_money_d1 =  pd.pivot_table(df_mob_total[df_mob_total.overdue_days_1==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d1 = vintage_money_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_money_d1 = vintage_money_d1.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d1['采购金额'] = df_mon["purchase_amount"]
# vintage_divide_money_d1[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [177]:
vintage_money_d4 =  pd.pivot_table(df_mob_total[df_mob_total.overdue_days_4==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4 = vintage_money_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d4['采购金额'] = df_mon['采购金额']
# vintage_money_d4
vintage_divide_money_d4 = vintage_money_d4.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d4[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,9.96%,27.27%,32.59%,34.52%,36.02%,35.77%,35.77%,35.77%,36.70%,33.76%,29.57%,29.57%
2022-08,358451.00,16.55%,25.37%,31.30%,31.87%,33.05%,35.44%,34.51%,35.92%,34.03%,30.79%,27.46%,26.76%
2022-09,277982.75,9.02%,13.86%,17.67%,22.33%,26.23%,24.21%,23.12%,22.98%,22.14%,17.88%,15.98%,15.98%
2022-10,249543.09,4.64%,5.86%,8.64%,13.66%,20.66%,17.70%,15.88%,18.33%,21.14%,16.99%,17.03%,15.04%
2022-11,344258.81,8.23%,11.76%,9.96%,14.05%,12.59%,19.69%,18.74%,15.33%,15.93%,14.96%,12.12%,12.12%
2022-12,566276.59,3.12%,7.69%,12.62%,17.67%,16.18%,15.47%,19.37%,19.45%,19.83%,17.56%,15.50%,13.89%
2023-01,1533459.00,5.21%,10.18%,11.82%,13.05%,16.01%,18.09%,17.41%,17.96%,17.77%,16.52%,14.11%,13.97%
2023-02,2181552.82,8.92%,13.29%,16.46%,17.30%,19.79%,20.32%,20.31%,20.66%,19.11%,18.18%,15.81%,15.55%
2023-03,2079408.00,7.51%,12.48%,15.23%,18.90%,20.36%,22.72%,21.61%,20.70%,20.84%,20.28%,17.84%,17.59%
2023-04,4317651.50,3.76%,8.09%,12.37%,14.29%,15.98%,17.53%,17.94%,18.30%,18.30%,17.77%,16.03%,15.77%


### 租物渠道

In [178]:

vintage_money_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_zm = vintage_money_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_zm = vintage_money_d4_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d4_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,24.49%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,6.57%,nan%,5.58%,5.58%,5.58%,17.43%,11.05%,11.05%,9.46%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,18.06%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,7.46%,3.90%,3.90%,3.90%,11.76%,14.42%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,1.87%,5.50%,7.13%,5.24%,5.37%,7.67%,6.46%,7.13%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,1.39%,1.39%,3.66%,4.42%,9.44%,9.89%,11.21%,12.00%,12.40%,13.37%,12.00%,12.00%
2023-04,1628122.00,3.16%,6.25%,8.90%,9.98%,10.79%,14.34%,14.71%,15.34%,15.52%,15.54%,14.30%,14.24%
2023-05,905764.00,nan%,3.93%,7.87%,7.83%,8.69%,7.89%,12.07%,13.42%,14.05%,12.78%,12.71%,12.15%
2023-06,1710725.00,3.20%,6.89%,10.98%,13.40%,14.62%,16.14%,17.84%,19.57%,19.17%,19.99%,18.13%,17.89%


### 搜索渠道

In [179]:

vintage_money_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_ss = vintage_money_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_ss = vintage_money_d4_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d4_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,9.59%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,43.42%,38.74%,33.82%,33.82%
2022-08,145114.00,14.92%,20.33%,25.37%,24.74%,24.74%,28.53%,30.37%,34.51%,27.74%,24.82%,21.85%,21.85%
2022-09,67126.34,37.34%,48.35%,54.03%,54.03%,53.64%,52.86%,48.35%,52.07%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,nan%,14.09%,14.09%,29.29%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,32.13%,39.16%,32.13%,38.13%,38.13%,42.44%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,2.70%,10.69%,15.08%,18.90%,19.41%,20.50%,22.14%,23.64%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,5.66%,10.83%,13.43%,12.61%,15.16%,16.94%,16.82%,17.23%,16.64%,15.20%,13.11%,12.89%
2023-02,1369414.82,11.35%,16.40%,18.65%,20.33%,22.87%,24.47%,24.87%,24.55%,23.02%,21.58%,18.64%,18.50%
2023-03,1002057.00,12.23%,19.15%,24.12%,27.42%,27.63%,30.59%,28.41%,26.66%,26.05%,25.23%,22.25%,21.99%
2023-04,1275193.00,4.29%,11.11%,16.07%,16.84%,18.39%,17.32%,19.71%,19.20%,19.60%,18.97%,17.11%,16.57%


### 抖音渠道

In [180]:

vintage_money_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_dy = vintage_money_d4_dy[['mob_1','mob_3']]  #'下单月份','mob_0','mob_1',,'mob_2',

vintage_divide_money_d4_dy = vintage_money_d4_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d4_dy[['采购金额','mob_1','mob_3']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'


MOB,采购金额,mob_1,mob_3
2024-04,15930.0,nan%,24.71%
2024-05,12690.0,nan%,nan%
2024-06,220205.0,14.35%,19.06%
2024-07,443358.0,12.57%,25.12%
2024-08,271022.0,17.48%,nan%
2024-09,30173.0,15.33%,nan%
2024-11,9865.0,nan%,nan%


## 金额vintage 7+ 历史最大逾期天数 口径 

### 总体

In [181]:
vintage_money_d7 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_7==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d7
vintage_divide_money_d7 = vintage_money_d7.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,3.56%,27.27%,32.59%,34.52%,36.02%,35.77%,35.77%,35.77%,36.70%,33.76%,29.57%,29.57%
2022-08,358451.00,12.55%,25.37%,31.30%,31.87%,33.05%,33.94%,34.51%,35.92%,34.03%,30.79%,27.46%,26.76%
2022-09,277982.75,9.02%,13.86%,17.67%,19.55%,26.23%,24.21%,23.12%,21.79%,22.14%,17.88%,15.98%,15.98%
2022-10,249543.09,4.64%,5.86%,6.79%,13.66%,20.66%,17.70%,15.88%,17.74%,21.14%,16.99%,17.03%,15.04%
2022-11,344258.81,8.23%,11.76%,9.96%,11.50%,11.50%,17.40%,18.74%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,1.55%,7.69%,11.27%,17.67%,14.94%,15.47%,19.37%,19.45%,19.83%,17.56%,15.50%,13.89%
2023-01,1533459.00,4.73%,9.65%,11.82%,13.05%,14.71%,17.91%,16.52%,17.61%,17.77%,16.59%,14.11%,13.97%
2023-02,2181552.82,8.47%,12.42%,15.86%,17.16%,19.34%,19.75%,20.21%,20.46%,18.94%,18.04%,15.81%,15.55%
2023-03,2079408.00,5.51%,11.60%,13.70%,18.18%,19.79%,21.28%,20.68%,20.26%,20.84%,20.28%,17.84%,17.59%
2023-04,4317651.50,3.39%,7.28%,11.50%,13.76%,15.53%,17.30%,17.51%,18.04%,18.17%,17.64%,16.03%,15.77%


### 租物渠道

In [182]:

vintage_money_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_zm = vintage_money_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_zm = vintage_money_d7_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d7_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,24.49%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,6.57%,nan%,5.58%,5.58%,5.58%,17.43%,11.05%,11.05%,9.46%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,18.06%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,nan%,3.90%,3.90%,3.90%,11.76%,11.76%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,nan%,3.76%,3.76%,3.76%,5.37%,6.45%,5.37%,7.13%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,1.39%,1.39%,3.66%,4.42%,9.44%,8.22%,9.63%,12.00%,12.40%,13.37%,12.00%,12.00%
2023-04,1628122.00,3.16%,5.80%,8.23%,9.98%,10.79%,14.34%,14.37%,15.34%,15.52%,15.54%,14.30%,14.24%
2023-05,905764.00,nan%,3.93%,7.81%,7.83%,7.76%,7.89%,12.07%,12.95%,14.05%,12.44%,12.52%,12.15%
2023-06,1710725.00,2.22%,5.61%,10.19%,12.45%,14.02%,15.53%,17.84%,18.51%,19.17%,19.81%,17.98%,17.89%


### 搜索渠道

In [183]:

vintage_money_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_ss = vintage_money_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_ss = vintage_money_d7_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d7_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,9.59%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,43.42%,38.74%,33.82%,33.82%
2022-08,145114.00,14.92%,20.33%,25.37%,24.74%,24.74%,28.53%,30.37%,34.51%,27.74%,24.82%,21.85%,21.85%
2022-09,67126.34,37.34%,48.35%,54.03%,54.03%,53.64%,52.86%,48.35%,52.07%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,nan%,14.09%,14.09%,29.29%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,32.13%,39.16%,32.13%,38.13%,38.13%,42.44%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,2.70%,10.69%,15.08%,18.90%,19.41%,20.50%,22.14%,23.64%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,4.87%,10.83%,13.43%,12.61%,13.05%,16.65%,15.68%,16.67%,16.64%,15.32%,13.11%,12.89%
2023-02,1369414.82,10.92%,15.28%,18.20%,20.33%,22.15%,23.75%,24.87%,24.55%,23.02%,21.37%,18.64%,18.50%
2023-03,1002057.00,9.22%,18.37%,22.38%,26.06%,27.08%,29.83%,27.94%,26.13%,26.05%,25.23%,22.25%,21.99%
2023-04,1275193.00,3.04%,8.93%,15.09%,15.85%,17.31%,16.74%,18.70%,18.92%,19.60%,18.75%,17.11%,16.57%


## 金额vintage 15+ 历史最大逾期天数 口径 

### 总体

In [184]:
vintage_money_d15 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_15==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份',
# vintage_money_d15
vintage_divide_money_d15 = vintage_money_d15.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,nan%,21.60%,32.59%,32.59%,32.59%,34.34%,35.77%,35.77%,35.77%,33.72%,29.57%,29.57%
2022-08,358451.00,7.07%,17.39%,23.88%,26.96%,31.27%,33.94%,34.51%,34.74%,32.95%,30.79%,27.46%,26.76%
2022-09,277982.75,2.99%,11.68%,17.67%,17.29%,24.06%,24.21%,23.12%,21.79%,21.01%,17.88%,15.98%,15.98%
2022-10,249543.09,4.64%,5.86%,6.79%,13.66%,18.55%,17.70%,15.88%,17.74%,21.14%,16.99%,17.03%,15.04%
2022-11,344258.81,5.56%,11.76%,9.96%,11.50%,11.50%,15.51%,18.74%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,nan%,7.69%,8.54%,17.67%,14.64%,14.56%,18.43%,17.55%,19.83%,17.56%,15.50%,13.89%
2023-01,1533459.00,3.94%,8.89%,9.81%,12.69%,14.36%,17.61%,16.22%,17.33%,17.57%,16.37%,14.11%,13.97%
2023-02,2181552.82,5.43%,10.18%,14.82%,14.83%,18.70%,18.68%,19.22%,19.99%,18.94%,17.92%,15.81%,15.55%
2023-03,2079408.00,3.94%,9.69%,11.96%,15.70%,17.20%,20.12%,19.65%,19.59%,20.10%,20.04%,17.84%,17.59%
2023-04,4317651.50,2.30%,5.86%,9.97%,12.37%,14.09%,16.31%,16.81%,17.15%,17.90%,16.92%,16.05%,15.77%


### 租物渠道

In [185]:

vintage_money_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_zm = vintage_money_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_zm = vintage_money_d15_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d15_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,nan%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,6.57%,nan%,5.58%,5.58%,5.58%,17.43%,11.05%,11.05%,9.46%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,18.06%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,nan%,3.90%,3.90%,3.90%,11.76%,11.76%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,nan%,3.76%,3.76%,3.76%,5.37%,6.45%,5.37%,7.13%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,nan%,1.39%,2.46%,3.34%,4.96%,8.22%,9.63%,11.21%,11.72%,13.37%,12.00%,12.00%
2023-04,1628122.00,2.26%,4.10%,7.14%,8.78%,10.14%,13.03%,13.74%,14.14%,15.52%,15.15%,14.30%,14.24%
2023-05,905764.00,nan%,2.62%,7.81%,7.83%,7.76%,7.64%,12.07%,12.35%,13.16%,12.15%,12.52%,12.15%
2023-06,1710725.00,1.36%,5.25%,8.77%,11.67%,13.68%,15.14%,17.31%,18.04%,18.83%,19.15%,17.98%,17.89%


### 搜索渠道

In [186]:

vintage_money_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_ss = vintage_money_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_ss = vintage_money_d15_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d15_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,nan%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,38.74%,33.82%,33.82%
2022-08,145114.00,11.41%,11.41%,17.03%,17.03%,24.74%,28.53%,30.37%,31.60%,27.74%,24.82%,21.85%,21.85%
2022-09,67126.34,12.40%,48.35%,54.03%,54.03%,53.64%,52.86%,48.35%,52.07%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,nan%,14.09%,14.09%,29.29%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,21.72%,39.16%,32.13%,38.13%,38.13%,42.44%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,nan%,10.69%,12.35%,18.90%,18.90%,18.90%,20.50%,20.34%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,3.59%,9.60%,10.18%,12.61%,12.49%,16.16%,15.19%,16.21%,16.31%,14.97%,13.11%,12.89%
2023-02,1369414.82,6.08%,12.93%,17.60%,17.50%,21.98%,22.43%,23.97%,24.39%,23.02%,21.17%,18.64%,18.50%
2023-03,1002057.00,7.36%,15.25%,19.21%,23.98%,24.81%,27.43%,26.27%,25.46%,25.62%,24.89%,22.25%,21.99%
2023-04,1275193.00,2.04%,8.03%,11.87%,13.70%,14.73%,16.09%,17.85%,17.79%,19.09%,17.44%,17.16%,16.57%


## 金额vintage 31+ 历史最大逾期天数 口径 

### 总体

In [187]:
vintage_money_d31 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_31==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31 = vintage_money_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_1',
# vintage_money_d31
vintage_divide_money_d31 = vintage_money_d31.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d31[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,21.60%,32.59%,32.59%,32.59%,34.34%,35.77%,35.77%,35.77%,33.72%,29.57%,29.57%
2022-08,358451.00,15.13%,21.76%,25.99%,30.08%,30.94%,32.81%,33.91%,31.77%,30.79%,26.93%,26.76%
2022-09,277982.75,9.02%,13.86%,16.86%,22.23%,23.12%,23.12%,20.90%,21.01%,17.88%,15.98%,15.98%
2022-10,249543.09,5.86%,5.70%,8.64%,13.66%,15.38%,11.91%,15.88%,15.58%,16.99%,15.27%,15.04%
2022-11,344258.81,10.03%,9.96%,11.50%,11.50%,12.59%,15.24%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,4.61%,6.04%,14.96%,14.64%,14.56%,17.27%,17.55%,19.03%,17.56%,15.50%,13.89%
2023-01,1533459.00,5.06%,7.87%,9.96%,12.06%,15.83%,14.89%,16.16%,17.11%,16.22%,14.04%,13.97%
2023-02,2181552.82,7.52%,11.40%,14.02%,16.89%,17.24%,18.26%,19.20%,18.54%,17.60%,15.73%,15.55%
2023-03,2079408.00,7.58%,10.76%,14.38%,16.78%,18.69%,18.62%,19.48%,19.65%,19.60%,17.50%,17.59%
2023-04,4317651.50,3.23%,6.59%,9.60%,12.14%,14.01%,15.60%,16.27%,17.30%,16.72%,15.90%,15.77%


### 租物渠道

In [188]:

vintage_money_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_zm = vintage_money_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d31_zm = vintage_money_d31_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d31_zm[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,24.49%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,nan%,nan%,5.58%,5.58%,5.58%,nan%,11.05%,nan%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,3.90%,3.90%,3.90%,11.76%,11.76%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,3.76%,3.76%,3.76%,5.37%,5.37%,5.37%,5.37%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,1.39%,1.39%,2.46%,4.96%,7.68%,9.35%,11.21%,10.77%,12.20%,12.00%,12.00%
2023-04,1628122.00,2.73%,5.47%,7.32%,9.24%,10.68%,12.11%,13.27%,14.99%,14.54%,14.00%,14.24%
2023-05,905764.00,nan%,3.15%,6.05%,6.70%,7.64%,9.31%,10.54%,12.47%,12.15%,11.58%,12.15%
2023-06,1710725.00,2.70%,6.06%,9.94%,11.64%,13.59%,15.22%,16.51%,17.98%,18.39%,17.73%,17.89%


### 搜索渠道

In [189]:

vintage_money_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_ss = vintage_money_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d31_ss = vintage_money_d31_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d31_ss[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,38.74%,33.82%,33.82%
2022-08,145114.00,11.41%,17.03%,17.03%,24.74%,24.74%,28.12%,29.55%,25.91%,24.82%,21.85%,21.85%
2022-09,67126.34,37.34%,48.35%,54.03%,53.64%,48.35%,48.35%,48.35%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,14.09%,14.09%,14.09%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,39.16%,32.13%,38.13%,38.13%,38.13%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,5.31%,7.99%,16.75%,18.90%,18.90%,20.50%,20.34%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,4.84%,7.49%,10.26%,10.69%,14.14%,13.85%,15.22%,15.68%,14.76%,12.99%,12.89%
2023-02,1369414.82,9.23%,13.37%,16.32%,20.14%,20.41%,22.52%,23.40%,22.41%,20.91%,18.50%,18.50%
2023-03,1002057.00,12.37%,17.71%,23.54%,24.46%,26.12%,24.71%,25.43%,25.13%,24.37%,21.75%,21.99%
2023-04,1275193.00,3.75%,7.34%,12.21%,14.32%,14.59%,16.28%,16.93%,18.46%,17.44%,17.09%,16.57%


# 以下为 租后表分析部分 

## 每月还款状态 sheet 数据输出

## 每个下单月份的金额口径的当前逾期率

In [190]:
dfzh3["下单月份"] = dfzh3['search_time']
dfzh3["未回收成本"] = dfzh3['unrecovered_cost']
dfzh3["逾期类型"] = dfzh3['overdue_type']
dfzh3["采购金额"] = dfzh3['purchase_amount']

In [191]:
df_zt = dfzh3[dfzh3["未回收成本"]>0]    #.groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")

df_zt['M0'] = np.where(df_zt.逾期类型=='M0',df_zt["未回收成本"],0)
df_zt['M1'] = np.where(df_zt.逾期类型=='M1',df_zt["未回收成本"],0)
df_zt['M2'] = np.where(df_zt.逾期类型=='M2',df_zt["未回收成本"],0)
df_zt['M3'] = np.where(df_zt.逾期类型=='M3',df_zt["未回收成本"],0)
df_zt['M4'] = np.where(df_zt.逾期类型=='M4',df_zt["未回收成本"],0)
df_zt['M5'] = np.where(df_zt.逾期类型=='M5',df_zt["未回收成本"],0)
df_zt['M6'] = np.where(df_zt.逾期类型=='M6',df_zt["未回收成本"],0)
dfyq = df_zt.groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=dfzh3[dfzh3["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
dfyq1=pd.merge(dfyq,dfzh3.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']]
# dfyq1.iloc[:,1:]#

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-07,0.00,0.00,0.00,0.00,37996.24,37996.24,1.254050e+05,30.30%,30.30%,30.30%,30.30%
2022-08,0.00,0.00,0.00,0.00,104969.98,104969.98,3.584510e+05,29.28%,29.28%,29.28%,29.28%
2022-09,0.00,0.00,0.00,0.00,53433.42,53433.42,2.779828e+05,19.22%,19.22%,19.22%,19.22%
2022-10,0.00,0.00,0.00,0.00,43583.45,43583.45,2.495431e+05,17.47%,17.47%,17.47%,17.47%
2022-11,0.00,0.00,0.00,0.00,48077.70,48077.70,3.442588e+05,13.97%,13.97%,13.97%,13.97%
2022-12,0.00,0.00,0.00,0.00,94904.60,94904.60,5.662766e+05,16.76%,16.76%,16.76%,16.76%
2023-01,0.00,0.00,0.00,1737.00,247277.25,249014.25,1.533459e+06,16.24%,16.24%,16.24%,16.13%
2023-02,0.00,0.00,0.00,0.00,363330.54,363330.54,2.183083e+06,16.64%,16.64%,16.64%,16.64%
2023-03,0.00,0.00,0.00,0.00,406344.16,406344.16,2.091200e+06,19.43%,19.43%,19.43%,19.43%


## 某渠道每月金额口径当前逾期率

In [192]:
df_zw = dfzh3[dfzh3["归属渠道"]=="芝麻租物"]#[df[df["归属渠道"]=="搜索渠道"]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-09,0.00,0.00,0.00,0.00,6668.60,6668.60,2.474268e+04,26.95%,26.95%,26.95%,26.95%
2022-10,0.00,0.00,0.00,0.00,4295.60,4295.60,4.883500e+04,8.80%,8.80%,8.80%,8.80%
2022-11,0.00,0.00,0.00,0.00,9771.79,9771.79,6.873022e+04,14.22%,14.22%,14.22%,14.22%
2022-12,0.00,0.00,0.00,0.00,1255.20,1255.20,7.657000e+03,16.39%,16.39%,16.39%,16.39%
2023-01,0.00,0.00,0.00,0.00,10493.55,10493.55,1.100700e+05,9.53%,9.53%,9.53%,9.53%
2023-02,0.00,0.00,0.00,0.00,15759.76,15759.76,2.081490e+05,7.57%,7.57%,7.57%,7.57%
2023-03,0.00,0.00,0.00,0.00,49946.37,49946.37,3.807040e+05,13.12%,13.12%,13.12%,13.12%
2023-04,0.00,0.00,0.00,237.12,309963.84,310200.96,1.628122e+06,19.05%,19.05%,19.05%,19.04%
2023-05,0.00,0.00,0.00,7615.21,129454.18,137069.39,9.066490e+05,15.12%,15.12%,15.12%,14.28%


In [193]:
df_zw = dfzh3[dfzh3["归属渠道"]=="搜索渠道"]  #df_zw=df[df["渠道名称"]=="芝麻租物"]#
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-07,0.00,0.00,0.00,0.00,16668.37,16668.37,46581.00,35.78%,35.78%,35.78%,35.78%
2022-08,0.00,0.00,0.00,0.00,34705.96,34705.96,145114.00,23.92%,23.92%,23.92%,23.92%
2022-09,0.00,0.00,0.00,0.00,31483.92,31483.92,67126.34,46.90%,46.90%,46.90%,46.90%
2022-10,0.00,0.00,0.00,0.00,5281.00,5281.00,44442.63,11.88%,11.88%,11.88%,11.88%
2022-11,0.00,0.00,0.00,0.00,27591.41,27591.41,88153.59,31.30%,31.30%,31.30%,31.30%
2022-12,0.00,0.00,0.00,0.00,57221.20,57221.20,324164.59,17.65%,17.65%,17.65%,17.65%
2023-01,0.00,0.00,0.00,1737.00,151808.98,153545.98,944943.00,16.25%,16.25%,16.25%,16.07%
2023-02,0.00,0.00,0.00,0.00,255574.54,255574.54,1369414.82,18.66%,18.66%,18.66%,18.66%
2023-03,0.00,0.00,0.00,0.00,241497.40,241497.40,1002057.00,24.10%,24.10%,24.10%,24.10%


In [194]:
df_zw = dfzh3[dfzh3["归属渠道"]=="抖音渠道"]  #df_zw=df[df["渠道名称"]=="芝麻租物"]#
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2024-04,0.00,655.85,0.0,3932.0,0.0,4587.85,15930.0,28.80%,28.80%,24.68%,0.00%
2024-05,0.00,0.00,0.0,0.0,0.0,0.00,12690.0,0.00%,0.00%,0.00%,0.00%
2024-06,12312.32,9522.57,3152.0,31900.0,0.0,56886.89,220205.0,25.83%,20.24%,14.49%,0.00%
2024-07,23358.98,23385.19,34846.9,45384.0,0.0,126975.07,443358.0,28.64%,23.37%,10.24%,0.00%
2024-08,8031.00,23679.36,37125.0,0.0,0.0,68835.36,271022.0,25.40%,22.44%,0.00%,0.00%
2024-09,2741.00,4625.00,0.0,0.0,0.0,7366.00,30173.0,24.41%,15.33%,0.00%,0.00%
2024-11,0.00,0.00,0.0,0.0,0.0,0.00,9865.0,0.00%,0.00%,0.00%,0.00%
总计,46443.30,61867.97,75123.9,81216.0,0.0,264651.17,1003243.0,26.38%,21.75%,8.10%,0.00%


## 总体订单口径每月当前逾期率

In [195]:
dfzh3["还款状态"] = dfzh3["rembursement_status"]

In [196]:
#总体每月当前逾期率 cpd1
df0 = pd.crosstab(dfzh3["下单月份"],dfzh3["还款状态"],margins=True)
df0["逾期率"]=df0["已逾期"]/df0[["已买断","已完成","已逾期","续租中","还款中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["买断率"]=df0["已买断"]/df0["All"]
df0["买断率"]=df0["买断率"].apply(lambda x:format(x,".2%"))
df0["归还率"]=(df0["已完成"]+df0["检测中"])/df0["All"]
df0["归还率"]=df0["归还率"].apply(lambda x:format(x,".2%"))
df0["续租率"]=df0["续租中"]/df0["All"]
df0["续租率"]=df0["续租率"].apply(lambda x:format(x,".2%"))
df0#[["已买断","All","买断率"]]#[14:]

还款状态,已买断,已完成,已逾期,未到首期还款日,检测中,续租中,还款中,All,逾期率,买断率,归还率,续租率
下单月份,,,,,,,,,,,,
2022-07,9,1,8,0,0,3,0,21,38.10%,42.86%,4.76%,14.29%
2022-08,20,3,30,0,1,1,0,55,54.55%,36.36%,7.27%,1.82%
2022-09,17,6,20,0,0,1,0,44,45.45%,38.64%,13.64%,2.27%
2022-10,18,6,12,0,1,4,0,41,29.27%,43.90%,17.07%,9.76%
2022-11,21,7,13,0,0,2,0,43,30.23%,48.84%,16.28%,4.65%
2022-12,34,4,24,0,0,4,0,66,36.36%,51.52%,6.06%,6.06%
2023-01,104,20,91,0,3,3,0,221,41.18%,47.06%,10.41%,1.36%
2023-02,155,48,118,0,1,10,1,333,35.44%,46.55%,14.71%,3.00%
2023-03,135,43,122,0,3,13,0,316,38.61%,42.72%,14.56%,4.11%


In [197]:
dfzh3["续租期数"] = dfzh3["relet_periods"] 
dfzh3["已还期数"] = dfzh3["paid_periods"] 


In [198]:
# 以下代码按顺序运行完到到期状态数据
# pd.crosstab(dfzh1["下单月份"],dfzh1["已还期数"],margins=True)
df_yq = dfzh3[dfzh3["还款状态"]=="已逾期"]

# df_yq=df[df["还款状态"]=="已逾期"]
# pd.crosstab(df_yq["下单月份"],df_yq["续租期数"],margins=True)

df_xz = dfzh3[dfzh3["续租期数"]>0]
# pd.crosstab(df_xz["下单月份"],df_xz["还款状态"],margins=True)

df_timeup = dfzh3[dfzh3["已还期数"]==12]


df_timeon = dfzh3[dfzh3["已还期数"]<12]
df_timeon_1 = pd.crosstab(df_timeon["下单月份"],df_timeon["还款状态"],margins=True).rename(columns={"已买断":"提前买断","已完成":"提前归还","已逾期":"租期逾期"})
df_timeup_xz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]>0)]
df_timeup_xz_1 = pd.crosstab(df_timeup_xz["下单月份"],df_timeup_xz["还款状态"],margins=True).rename(columns={"已买断":"续租后买断","已完成":"续租后归还","已逾期":"续租后逾期","检测中":"续租归还检测中"})
df_timeup_nxz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]<1)]
df_timeup_nxz_1 = pd.crosstab(df_timeup_nxz["下单月份"],df_timeup_nxz["还款状态"],margins=True).rename(columns={"已买断":"到期买断","已完成":"到期归还","已逾期":"到期逾期","检测中":"到期归还检测中","续租中":"待续租","还款中":"已付总租金待归还"})


In [199]:
#以下为分别统计每个下单月的订单在12期租期内租期逾期、提前买断、提前归还的的订单数，
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单数，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单数
# df_timeon_1.drop(columns=['检测中','续租中'],inplace = True)
df_due = pd.merge(pd.merge(df_timeon_1,df_timeup_nxz_1,left_on="下单月份",right_on="下单月份",how="left"),df_timeup_xz_1,left_on="下单月份",right_on="下单月份",how="left")
df_due=df_due.fillna(0)
df_due["合计"]=df_due["All_x"]+df_due["All_y"]+df_due["All"]
#df_due["合计"]=df_due["合计"].astype("int")
df_due.drop(columns=["All_x","All_y","All"],axis=1,inplace=True)
df_due=df_due.astype("int")
df_due#[["提前买断","到期买断","续租后买断"]]#[14:]#[14:]

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,0,0,8,0,0,0,6,1,0,0,0,0,3,0,0,0,3,21
2022-08,2,0,21,0,0,0,7,2,7,0,0,0,11,1,2,1,1,55
2022-09,4,2,10,0,0,0,4,2,5,0,0,0,9,2,5,0,1,44
2022-10,1,1,8,0,0,0,8,2,1,1,0,0,9,3,3,0,4,41
2022-11,5,0,8,0,0,0,6,6,1,0,0,0,10,1,4,0,2,43
2022-12,8,2,16,0,0,0,13,2,6,0,0,0,13,0,2,0,4,66
2023-01,17,4,64,0,0,0,42,16,14,3,0,0,45,0,13,0,3,221
2023-02,24,5,73,0,0,1,90,43,30,1,0,0,41,0,15,0,10,333
2023-03,24,11,85,0,0,0,71,32,25,3,0,0,40,0,12,0,13,316


In [200]:
#以下为分别统计每个下单月的订单在12期租期内租期逾期、提前买断、提前归还的的订单占比，
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单占比，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单占比
#该代码需先运行上述代码方不会报错

df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]


还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,0.00%,0.00%,38.10%,0.00%,0.00%,0.00%,28.57%,4.76%,0.00%,0.00%,0.00%,0.00%,14.29%,0.00%,0.00%,0.00%,14.29%,100.00%
2022-08,3.64%,0.00%,38.18%,0.00%,0.00%,0.00%,12.73%,3.64%,12.73%,0.00%,0.00%,0.00%,20.00%,1.82%,3.64%,1.82%,1.82%,100.00%
2022-09,9.09%,4.55%,22.73%,0.00%,0.00%,0.00%,9.09%,4.55%,11.36%,0.00%,0.00%,0.00%,20.45%,4.55%,11.36%,0.00%,2.27%,100.00%
2022-10,2.44%,2.44%,19.51%,0.00%,0.00%,0.00%,19.51%,4.88%,2.44%,2.44%,0.00%,0.00%,21.95%,7.32%,7.32%,0.00%,9.76%,100.00%
2022-11,11.63%,0.00%,18.60%,0.00%,0.00%,0.00%,13.95%,13.95%,2.33%,0.00%,0.00%,0.00%,23.26%,2.33%,9.30%,0.00%,4.65%,100.00%
2022-12,12.12%,3.03%,24.24%,0.00%,0.00%,0.00%,19.70%,3.03%,9.09%,0.00%,0.00%,0.00%,19.70%,0.00%,3.03%,0.00%,6.06%,100.00%
2023-01,7.69%,1.81%,28.96%,0.00%,0.00%,0.00%,19.00%,7.24%,6.33%,1.36%,0.00%,0.00%,20.36%,0.00%,5.88%,0.00%,1.36%,100.00%
2023-02,7.21%,1.50%,21.92%,0.00%,0.00%,0.30%,27.03%,12.91%,9.01%,0.30%,0.00%,0.00%,12.31%,0.00%,4.50%,0.00%,3.00%,100.00%
2023-03,7.59%,3.48%,26.90%,0.00%,0.00%,0.00%,22.47%,10.13%,7.91%,0.95%,0.00%,0.00%,12.66%,0.00%,3.80%,0.00%,4.11%,100.00%


In [201]:
# 以下为每月预收租金期数占比
dfzh3["预收期数"] = dfzh3["advance_periods"] 
df_ys=pd.crosstab(dfzh3["下单月份"],dfzh3["预收期数"],margins=True)
df_ys.divide(df_ys["All"],axis=0).applymap(lambda x:format(x,".2%"))


预收期数,0,1,2,3,4,5,6,7,8,9,12,All
下单月份,,,,,,,,,,,,
2022-07,0.00%,9.52%,38.10%,47.62%,4.76%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-08,0.00%,1.82%,27.27%,49.09%,12.73%,5.45%,3.64%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-09,0.00%,0.00%,34.09%,45.45%,18.18%,2.27%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-10,2.44%,24.39%,41.46%,14.63%,12.20%,4.88%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-11,0.00%,11.63%,51.16%,16.28%,18.60%,2.33%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-12,0.00%,18.18%,42.42%,19.70%,18.18%,1.52%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2023-01,0.45%,16.29%,54.30%,19.00%,5.88%,2.26%,0.45%,0.45%,0.00%,0.00%,0.90%,100.00%
2023-02,0.00%,5.71%,60.96%,17.42%,8.41%,4.80%,1.80%,0.00%,0.30%,0.30%,0.30%,100.00%
2023-03,0.00%,15.51%,59.81%,11.08%,4.43%,3.16%,5.70%,0.00%,0.32%,0.00%,0.00%,100.00%


## 平台套餐统计

In [202]:
dfzh3['平台套餐'] = dfzh3['platform_package']
dfzh3['租赁方案'] = np.where(dfzh3.平台套餐=='租完即送','租完即送','租完归还/可买断')
dfzh3['租赁方案'].value_counts()

租赁方案
租完归还/可买断    23497
租完即送         2141
Name: count, dtype: int64

In [203]:

zl_class_df = pd.crosstab(dfzh3['下单月份'],dfzh3['租赁方案'],margins=True)
zl_class_df.divide(zl_class_df["All"],axis=0).applymap(lambda x:format(x,".2%"))


租赁方案,租完即送,租完归还/可买断,All
下单月份,,,
2022-07,28.57%,71.43%,100.00%
2022-08,16.36%,83.64%,100.00%
2022-09,2.27%,97.73%,100.00%
2022-10,0.00%,100.00%,100.00%
2022-11,6.98%,93.02%,100.00%
2022-12,12.12%,87.88%,100.00%
2023-01,19.46%,80.54%,100.00%
2023-02,9.01%,90.99%,100.00%
2023-03,14.87%,85.13%,100.00%


## 每月各渠道逾期率 sheet 数据输出

In [204]:
# df_fq_concat.describe().to_excel('F:/myfile/业务相关数据/df_desc.xlsx')

In [205]:
df_yq_fq = df_fq_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_yq_fq = df_yq_fq.reset_index()

df_yq_fq["FPD"] = df_yq_fq[2]
df_yq_fq.loc[:,"SPD"] = np.where(df_yq_fq[3]>df_yq_fq[2],df_yq_fq[3],df_yq_fq[2])
df_yq_fq.loc[:,"TPD"] = np.where(df_yq_fq[4]>df_yq_fq["SPD"],df_yq_fq[4],df_yq_fq["SPD"])
df_yq_fq.loc[:,"QPD"] = np.where(df_yq_fq[5]>df_yq_fq["TPD"],df_yq_fq[5],df_yq_fq["TPD"])

df_yq_fq[['order_id',2,3,4,'FPD','SPD','TPD','QPD']]

当前期数,order_id,2.0,3.0,4.0,FPD,SPD,TPD,QPD
0,8924,0.0,0.0,4.0,0.0,0.0,4.0,4.0
1,9514,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9600,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9683,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9699,8.0,5.0,9.0,8.0,8.0,9.0,9.0
...,...,...,...,...,...,...,...,...
25581,1765838,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25582,1765875,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25583,1765899,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25584,1765950,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [206]:
df_agr_1 =df_fq_concat[["order_id","当前期数","agr_1d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_1d_cum").reset_index()
df_agr_1 = df_agr_1.copy().add_suffix('_agr1')

df_agr_4 =df_fq_concat[["order_id","当前期数","agr_4d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_4d_cum").reset_index()
df_agr_4 = df_agr_4.copy().add_suffix('_agr4')
df_agr_7 =df_fq_concat[["order_id","当前期数","agr_7d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_7d_cum").reset_index()
df_agr_7 = df_agr_7.copy().add_suffix('_agr7')
df_agr_15 =df_fq_concat[["order_id","当前期数","agr_15d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_15d_cum").reset_index()
df_agr_15 = df_agr_15.copy().add_suffix('_agr15')
df_agr_30 =df_fq_concat[["order_id","当前期数","agr_30d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_30d_cum").reset_index()
df_agr_30 = df_agr_30.copy().add_suffix('_agr30')

In [207]:
df_agr_1

当前期数,order_id_agr1,1.0_agr1,2.0_agr1,3.0_agr1,4.0_agr1,5.0_agr1,6.0_agr1,7.0_agr1,8.0_agr1,9.0_agr1,10.0_agr1,11.0_agr1,12.0_agr1
0,8924,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,9514,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,9600,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,9683,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,9699,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25581,1765838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25582,1765875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25583,1765899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25584,1765950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [208]:
df_yq_total = pd.concat([df_yq_fq,df_agr_1,df_agr_4,df_agr_7,df_agr_15,df_agr_30],axis=1)
df_yq_total

当前期数,order_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,3.0_agr30,4.0_agr30,5.0_agr30,6.0_agr30,7.0_agr30,8.0_agr30,9.0_agr30,10.0_agr30,11.0_agr30,12.0_agr30
0,8924,0.0,0.0,0.0,4.0,3.0,3.0,2.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,9514,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,9600,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,9683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,9699,1.0,8.0,5.0,9.0,0.0,0.0,2.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25581,1765838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25582,1765875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25583,1765899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25584,1765950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [209]:
# 合并数据 
df = pd.merge(dfzh3,df_yq_total,on="order_id",how="left")
df.shape

(25638, 179)

In [210]:
df["客户标签fpd4"]=np.where(df["FPD"]>3,"逾期","正常")
df["客户标签fpd1"]=np.where(df["FPD"]>0,"逾期","正常")
df["客户标签fpd7"]=np.where(df["FPD"]>6,"逾期","正常")
df["客户标签fpd15"]=np.where(df["FPD"]>14,"逾期","正常")
df["客户标签spd4"]=np.where(df["SPD"]>3,"逾期","正常")
df["客户标签spd1"]=np.where(df["SPD"]>1,"逾期","正常")
df["客户标签spd15"]=np.where(df["SPD"]>14,"逾期","正常")
df["客户标签tpd15"]=np.where(df["TPD"]>14,"逾期","正常")

In [211]:
df["当前逾期天数"] = df["now_overdue_days"]
df["最长逾期天数"] = df["max_overdue_days"]


In [212]:
def getremark(a,b,c):
    if a>=c:
        return "逾期"
    elif a<c and b =="未到首期还款日":
        return "未到首期还款日"
    else:
        return "正常"
df["当前逾期1+"] = df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],1),axis=1)
df["当前逾期1+"].value_counts()

当前逾期1+
正常         17869
逾期          5539
未到首期还款日     2230
Name: count, dtype: int64

In [213]:
df["当前逾期4+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],4),axis=1)
df["当前逾期7+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],7),axis=1)
df["当前逾期15+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],15),axis=1)
df["当前逾期31+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],31),axis=1)
df["M12"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],366),axis=1)


df["历史逾期1+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],1),axis=1)
df["历史逾期7+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],7),axis=1)
df["历史逾期15+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],15),axis=1)
df["历史逾期31+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],31),axis=1)

In [214]:
# 当前逾期1+  按渠道名称
df_qdmc_4 = df[df["下单月份"]=="2024-05"]
df0=pd.crosstab(df_qdmc_4["channel_name"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0

当前逾期1+,正常,逾期,All,逾期率,lift
channel_name,,,,,
ocpc首页推荐交易,3,1,4,25.00%,1.4
付费流量(通过商家数字推广平台，灯火等投放的广告),26,2,28,7.14%,0.401
付费流量-灯火内测,1,0,1,0.00%,0.0
其他渠道场景渠道。,3,0,3,0.00%,0.0
分期乐1,2,0,2,0.00%,0.0
分期乐2,1,1,2,50.00%,2.81
富士康租机,14,5,19,26.32%,1.48
我的小程序入口,11,3,14,21.43%,1.2
搜索-租安卓手机,1,0,1,0.00%,0.0


In [215]:
# 当前逾期1+  按渠道名称
df_qdmc_4 = df[df["下单月份"]=="2024-06"]
df0=pd.crosstab(df_qdmc_4["channel_name"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift']]  # ,'未到首期还款日'

当前逾期1+,正常,逾期,All,逾期率,lift
channel_name,,,,,
618双重狂欢,1,0,1,0.00%,0.0
ocpc首页推荐交易,2,0,2,0.00%,0.0
付费流量(通过商家数字推广平台，灯火等投放的广告),51,7,58,12.07%,0.757
八派信息,9,3,12,25.00%,1.57
其他渠道场景渠道。,4,0,4,0.00%,0.0
分期乐1,1,0,1,0.00%,0.0
单人聊天会话中的小程序消息卡片（分享）,4,1,5,20.00%,1.25
富士康租机,6,4,10,40.00%,2.51
小程序商家消息（服务提醒）,4,0,4,0.00%,0.0


In [216]:
# 当前逾期1+  按渠道名称
df_qdmc_4 = df[df["下单月份"]=="2024-07"]
df0=pd.crosstab(df_qdmc_4["channel_name"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift','未到首期还款日']]  # ,'未到首期还款日'

当前逾期1+,正常,逾期,All,逾期率,lift,未到首期还款日
channel_name,,,,,,
ocpc首页推荐交易,10,1,11,9.09%,0.71,0
付费流量(通过商家数字推广平台，灯火等投放的广告),5,3,8,37.50%,2.93,0
付费灯火-蓝海词,4,1,5,20.00%,1.56,0
付费灯火-高频词,19,4,23,17.39%,1.36,0
其他渠道场景渠道。,5,0,5,0.00%,0.0,0
小程序商家消息（服务提醒）,2,1,3,33.33%,2.6,0
小程序快窗,3,0,3,0.00%,0.0,0
我的小程序入口,20,2,22,9.09%,0.71,0
搜索-3C数码租赁,4,1,5,20.00%,1.56,0


In [217]:
# 当前逾期1+ 按归属渠道
df_qdmc_4 = df[df["下单月份"]=="2024-05"]
df0=pd.crosstab(df_qdmc_4["归属渠道"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0

当前逾期1+,正常,逾期,All,逾期率,lift
归属渠道,,,,,
ocpc首页推荐交易,3,1,4,25.00%,1.4
付费流量(通过商家数字推广平台，灯火等投放的广告),26,2,28,7.14%,0.401
付费流量-灯火内测,1,0,1,0.00%,0.0
其他渠道场景渠道。,3,0,3,0.00%,0.0
分期乐1,2,0,2,0.00%,0.0
分期乐2,1,1,2,50.00%,2.81
富士康租机,14,5,19,26.32%,1.48
我的小程序入口,11,3,14,21.43%,1.2
抖音渠道,3,0,3,0.00%,0.0


In [218]:
# 当前逾期1+ 按归属渠道
df_qdmc_4=df[df["下单月份"]=="2024-06"]
df0=pd.crosstab(df_qdmc_4["归属渠道"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift']]

当前逾期1+,正常,逾期,All,逾期率,lift
归属渠道,,,,,
618双重狂欢,1,0,1,0.00%,0.0
ocpc首页推荐交易,2,0,2,0.00%,0.0
付费流量(通过商家数字推广平台，灯火等投放的广告),49,7,56,12.50%,0.784
八派信息,9,3,12,25.00%,1.57
其他渠道场景渠道。,3,0,3,0.00%,0.0
单人聊天会话中的小程序消息卡片（分享）,3,1,4,25.00%,1.57
富士康租机,6,3,9,33.33%,2.09
小程序商家消息（服务提醒）,1,0,1,0.00%,0.0
我的小程序入口,14,3,17,17.65%,1.11


In [219]:
# 当前逾期1+ 按归属渠道
df_qdmc_4=df[df["下单月份"]=="2024-07"]
df0=pd.crosstab(df_qdmc_4["归属渠道"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift','未到首期还款日']]

当前逾期1+,正常,逾期,All,逾期率,lift,未到首期还款日
归属渠道,,,,,,
ocpc首页推荐交易,10,0,10,0.00%,0.0,0
付费流量(通过商家数字推广平台，灯火等投放的广告),5,2,7,28.57%,2.23,0
付费灯火,22,5,27,18.52%,1.45,0
其他渠道场景渠道。,5,0,5,0.00%,0.0,0
小程序商家消息（服务提醒）,2,0,2,0.00%,0.0,0
小程序快窗,3,0,3,0.00%,0.0,0
我的小程序入口,17,0,17,0.00%,0.0,0
抖音渠道,62,44,106,41.51%,3.24,0
搜索渠道,363,70,434,16.17%,1.26,1


# 以下为每月总体、搜索渠道、芝麻租物渠道的逾期率和提升度横向对比

## 当前逾期1+

In [220]:
df0 = pd.crosstab(df["下单月份"],df["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})
# df0
df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期1+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期1+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期1+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

df0_4[["未到首期_总体","正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3   #[14:]#.iloc[:,:15]

当前逾期1+,未到首期_总体,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_会话,逾期率_会话,lift_会话,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_抖音,逾期率_抖音,lift_抖音
下单月份,,,,,,,,,,,,,,,,,
2022-07,0,12,9,21,42.86%,NaN,NaN,nan,NaN,NaN,nan,7,57.14%,1.33,NaN,NaN,nan
2022-08,0,23,32,55,58.18%,NaN,NaN,nan,NaN,NaN,nan,22,59.09%,1.02,NaN,NaN,nan
2022-09,0,23,21,44,47.73%,NaN,NaN,nan,5.0,40.00%,0.838,8,75.00%,1.57,NaN,NaN,nan
2022-10,0,27,14,41,34.15%,1.0,0.00%,0.0,10.0,30.00%,0.878,6,16.67%,0.488,NaN,NaN,nan
2022-11,0,30,13,43,30.23%,NaN,NaN,nan,10.0,30.00%,0.992,10,60.00%,1.98,NaN,NaN,nan
2022-12,0,42,24,66,36.36%,1.0,0.00%,0.0,2.0,50.00%,1.38,37,35.14%,0.966,NaN,NaN,nan
2023-01,0,127,94,221,42.53%,NaN,NaN,nan,18.0,27.78%,0.653,134,44.03%,1.04,NaN,NaN,nan
2023-02,0,214,119,333,35.74%,3.0,33.33%,0.933,39.0,15.38%,0.43,200,39.50%,1.11,NaN,NaN,nan
2023-03,0,191,125,316,39.56%,2.0,100.00%,2.53,62.0,25.81%,0.652,150,48.00%,1.21,NaN,NaN,nan


## 当前逾期4+

In [221]:
# 当前逾期4+
df0=pd.crosstab(df["下单月份"],df["当前逾期4+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期4+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期4+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期4+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
# df3.columns
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",


,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_会话,逾期率_会话,lift_会话,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_抖音,逾期率_抖音,lift_抖音
下单月份,,,,,,,,,,,,,,,,
2022-07,12,9,21,42.86%,NaN,NaN,nan,NaN,NaN,nan,7,57.14%,1.33,NaN,NaN,nan
2022-08,23,32,55,58.18%,NaN,NaN,nan,NaN,NaN,nan,22,59.09%,1.02,NaN,NaN,nan
2022-09,23,21,44,47.73%,NaN,NaN,nan,5.0,40.00%,0.838,8,75.00%,1.57,NaN,NaN,nan
2022-10,27,14,41,34.15%,1.0,0.00%,0.0,10.0,30.00%,0.878,6,16.67%,0.488,NaN,NaN,nan
2022-11,30,13,43,30.23%,NaN,NaN,nan,10.0,30.00%,0.992,10,60.00%,1.98,NaN,NaN,nan
2022-12,42,24,66,36.36%,1.0,0.00%,0.0,2.0,50.00%,1.38,37,35.14%,0.966,NaN,NaN,nan
2023-01,127,94,221,42.53%,NaN,NaN,nan,18.0,27.78%,0.653,134,44.03%,1.04,NaN,NaN,nan
2023-02,214,119,333,35.74%,3.0,33.33%,0.933,39.0,15.38%,0.43,200,39.50%,1.11,NaN,NaN,nan
2023-03,191,125,316,39.56%,2.0,100.00%,2.53,62.0,25.81%,0.652,150,48.00%,1.21,NaN,NaN,nan


## 当前逾期7+

In [222]:
# 当前逾期7+
df0=pd.crosstab(df["下单月份"],df["当前逾期7+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期7+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期7+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期7+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",


,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_会话,逾期率_会话,lift_会话,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_抖音,逾期率_抖音,lift_抖音
下单月份,,,,,,,,,,,,,,,,
2022-07,12,9,21,42.86%,NaN,NaN,nan,NaN,NaN,nan,7,57.14%,1.33,NaN,NaN,nan
2022-08,23,32,55,58.18%,NaN,NaN,nan,NaN,NaN,nan,22,59.09%,1.02,NaN,NaN,nan
2022-09,23,21,44,47.73%,NaN,NaN,nan,5.0,40.00%,0.838,8,75.00%,1.57,NaN,NaN,nan
2022-10,27,14,41,34.15%,1.0,0.00%,0.0,10.0,30.00%,0.878,6,16.67%,0.488,NaN,NaN,nan
2022-11,30,13,43,30.23%,NaN,NaN,nan,10.0,30.00%,0.992,10,60.00%,1.98,NaN,NaN,nan
2022-12,42,24,66,36.36%,1.0,0.00%,0.0,2.0,50.00%,1.38,37,35.14%,0.966,NaN,NaN,nan
2023-01,127,94,221,42.53%,NaN,NaN,nan,18.0,27.78%,0.653,134,44.03%,1.04,NaN,NaN,nan
2023-02,214,119,333,35.74%,3.0,33.33%,0.933,39.0,15.38%,0.43,200,39.50%,1.11,NaN,NaN,nan
2023-03,191,125,316,39.56%,2.0,100.00%,2.53,62.0,25.81%,0.652,150,48.00%,1.21,NaN,NaN,nan


## 当前逾期15+

In [223]:
# 当前逾期15+
df0=pd.crosstab(df["下单月份"],df["当前逾期15+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期15+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期15+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期15+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",



,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_会话,逾期率_会话,lift_会话,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_抖音,逾期率_抖音,lift_抖音
下单月份,,,,,,,,,,,,,,,,
2022-07,12,9,21,42.86%,NaN,NaN,nan,NaN,NaN,nan,7,57.14%,1.33,NaN,NaN,nan
2022-08,23,32,55,58.18%,NaN,NaN,nan,NaN,NaN,nan,22,59.09%,1.02,NaN,NaN,nan
2022-09,23,21,44,47.73%,NaN,NaN,nan,5.0,40.00%,0.838,8,75.00%,1.57,NaN,NaN,nan
2022-10,27,14,41,34.15%,1.0,0.00%,0.0,10.0,30.00%,0.878,6,16.67%,0.488,NaN,NaN,nan
2022-11,30,13,43,30.23%,NaN,NaN,nan,10.0,30.00%,0.992,10,60.00%,1.98,NaN,NaN,nan
2022-12,42,24,66,36.36%,1.0,0.00%,0.0,2.0,50.00%,1.38,37,35.14%,0.966,NaN,NaN,nan
2023-01,127,94,221,42.53%,NaN,NaN,nan,18.0,27.78%,0.653,134,44.03%,1.04,NaN,NaN,nan
2023-02,214,119,333,35.74%,3.0,33.33%,0.933,39.0,15.38%,0.43,200,39.50%,1.11,NaN,NaN,nan
2023-03,191,125,316,39.56%,2.0,100.00%,2.53,62.0,25.81%,0.652,150,48.00%,1.21,NaN,NaN,nan


## 客户标签fpd15

In [224]:
# 客户标签fpd15
df0=pd.crosstab(df["下单月份"],df["客户标签fpd15"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",
df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["客户标签fpd15"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))
df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["客户标签fpd15"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))
df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["客户标签fpd15"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]

# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",



,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_会话,逾期率_会话,lift_会话,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_抖音,逾期率_抖音,lift_抖音
下单月份,,,,,,,,,,,,,,,,
2022-07,19,2,21,9.52%,NaN,NaN,nan,NaN,NaN,nan,7,14.29%,1.5,NaN,NaN,nan
2022-08,48,7,55,12.73%,NaN,NaN,nan,NaN,NaN,nan,22,9.09%,0.714,NaN,NaN,nan
2022-09,41,3,44,6.82%,NaN,NaN,nan,5.0,0.00%,0.0,8,37.50%,5.5,NaN,NaN,nan
2022-10,38,3,41,7.32%,1.0,0.00%,0.0,10.0,10.00%,1.37,6,16.67%,2.28,NaN,NaN,nan
2022-11,39,4,43,9.30%,NaN,NaN,nan,10.0,0.00%,0.0,10,40.00%,4.3,NaN,NaN,nan
2022-12,60,6,66,9.09%,1.0,0.00%,0.0,2.0,0.00%,0.0,37,10.81%,1.19,NaN,NaN,nan
2023-01,206,15,221,6.79%,NaN,NaN,nan,18.0,0.00%,0.0,134,6.72%,0.99,NaN,NaN,nan
2023-02,304,29,333,8.71%,3.0,0.00%,0.0,39.0,7.69%,0.883,200,10.50%,1.21,NaN,NaN,nan
2023-03,291,25,316,7.91%,2.0,50.00%,6.32,62.0,1.61%,0.204,150,11.33%,1.43,NaN,NaN,nan


## 客户标签spd15

In [225]:
# 客户标签spd15
df0=pd.crosstab(df["下单月份"],df["客户标签spd15"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",
df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["客户标签spd15"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))
df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["客户标签spd15"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))
df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["客户标签spd15"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",


,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_会话,逾期率_会话,lift_会话,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_抖音,逾期率_抖音,lift_抖音
下单月份,,,,,,,,,,,,,,,,
2022-07,15,6,21,28.57%,NaN,NaN,nan,NaN,NaN,nan,7,42.86%,1.5,NaN,NaN,nan
2022-08,43,12,55,21.82%,NaN,NaN,nan,NaN,NaN,nan,22,18.18%,0.833,NaN,NaN,nan
2022-09,39,5,44,11.36%,NaN,NaN,nan,5.0,20.00%,1.76,8,50.00%,4.4,NaN,NaN,nan
2022-10,38,3,41,7.32%,1.0,0.00%,0.0,10.0,10.00%,1.37,6,16.67%,2.28,NaN,NaN,nan
2022-11,38,5,43,11.63%,NaN,NaN,nan,10.0,10.00%,0.86,10,40.00%,3.44,NaN,NaN,nan
2022-12,59,7,66,10.61%,1.0,0.00%,0.0,2.0,0.00%,0.0,37,13.51%,1.27,NaN,NaN,nan
2023-01,190,31,221,14.03%,NaN,NaN,nan,18.0,11.11%,0.792,134,15.67%,1.12,NaN,NaN,nan
2023-02,287,46,333,13.81%,3.0,0.00%,0.0,39.0,7.69%,0.557,200,17.00%,1.23,NaN,NaN,nan
2023-03,274,42,316,13.29%,2.0,50.00%,3.76,62.0,3.23%,0.243,150,19.33%,1.45,NaN,NaN,nan


# 每月前四期逾期率对比

In [226]:
df["fpd4"] = np.where((df["FPD"]>3) & (df['2_agr4'] ==1),1,0)
df["fpd7"] = np.where((df["FPD"]>6) & (df['2_agr7'] ==1),1,0)
df["fpd15"] = np.where((df["FPD"]>14) & (df['2_agr15'] ==1),1,0)
df["fpd30"] = np.where((df["FPD"]>29) & (df['2_agr30'] ==1),1,0)

df["spd4"]=np.where((df["SPD"]>3) & (df['3_agr4'] ==1),1,0)
df["spd7"]=np.where((df["SPD"]>6) & (df['3_agr7'] ==1),1,0)
df["spd15"]=np.where((df["SPD"]>14) & (df['3_agr15'] ==1),1,0)
df["spd30"]=np.where((df["SPD"]>29) & (df['3_agr30'] ==1),1,0)

df["tpd4"]=np.where((df["TPD"]>3) & (df['4_agr4'] ==1),1,0)
df["tpd7"]=np.where((df["TPD"]>6) & (df['4_agr7'] ==1),1,0)
df["tpd15"]=np.where((df["TPD"]>14) & (df['4_agr15'] ==1),1,0)
df["tpd30"]=np.where((df["TPD"]>29) & (df['4_agr30'] ==1),1,0)

df["qpd4"]=np.where((df["QPD"]>3) & (df['5_agr4'] ==1),1,0)
df["qpd7"]=np.where((df["QPD"]>6) & (df['5_agr7'] ==1),1,0)
df["qpd15"]=np.where((df["QPD"]>14) & (df['5_agr15'] ==1),1,0)
df["qpd30"]=np.where((df["QPD"]>29) & (df['5_agr30'] ==1),1,0)

df["fpd1"] = np.where((df["FPD"]>=1) & (df['2_agr1'] ==1),1,0)
df["spd1"]=np.where((df["SPD"]>=1) & (df['3_agr1'] ==1),1,0)
df["tpd1"]=np.where((df["TPD"]>=1) & (df['4_agr1'] ==1),1,0)
df["qpd1"]=np.where((df["QPD"]>=1) & (df['5_agr1'] ==1),1,0)

KeyError: '2_agr4'

In [ ]:
# df[(df['下单月份']=='2024-05')&(df["fpd15"]==1)][['订单号',2,'FPD','2_agr15',"fpd15"]]    #.sum()

## 总体

## 月度fstq

In [ ]:
df_tag4 = df.groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
import datetime 
Today= datetime.date.today()
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='xlsxwriter') as writer:    #, mode='a'
    # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq.to_excel(writer, sheet_name='mon_fstq', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_tag4 = df.groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
# ,'2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
# ,'3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
# ,'4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
# ,'5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})

# df_tag4

## 周度fstq

In [ ]:
# df['week_number'] = df['下单日期'].dt.isocalendar().week
df['week_start_date'] = df['下单日期'] - pd.to_timedelta(df['下单日期'].dt.weekday, unit='D')  

df['week_start_date']

In [ ]:
df_tag4 = df.groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# result[result.index>='2024-01-01']

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq.to_excel(writer, sheet_name='week_fstq', index=True)  # index=False 表示不导出行索引  

## 租物渠道

In [ ]:
df_tag4 = df[df.归属渠道=='芝麻租物'].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_zm = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_zm.to_excel(writer, sheet_name='mon_fstq_zm', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_tag4 = df[df.归属渠道=='芝麻租物'].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_zm = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# week_fstq_zm[week_fstq_zm.index>='2024-01-01']

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_zm.to_excel(writer, sheet_name='week_fstq_zm', index=True)  # index=False 表示不导出行索引  

## 搜索渠道

In [ ]:
df_tag4 = df[df.归属渠道=='搜索渠道'].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_ss = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_ss.to_excel(writer, sheet_name='mon_fstq_ss', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_tag4 = df[df.归属渠道=='搜索渠道'].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_ss = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# week_fstq_ss[week_fstq_ss.index>='2024-01-01']

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_ss.to_excel(writer, sheet_name='week_fstq_ss', index=True)  # index=False 表示不导出行索引  

## 抖音渠道

In [ ]:
df_tag4 = df[df.归属渠道=='抖音渠道'].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_dy = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# mon_fstq_dy

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_dy.to_excel(writer, sheet_name='mon_fstq_dy', index=True)  # index=False 表示不导出行索引  

In [ ]:
# 分周 
df_tag4 = df[df.归属渠道=='抖音渠道'].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_dy = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# week_fstq_dy

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_dy.to_excel(writer, sheet_name='week_fstq_dy', index=True)  # index=False 表示不导出行索引  

# 免审 fstq 表现

In [ ]:
sql = '''-- 免人审 
SELECT date(r.time) as risk_date 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.grade_msg') end,'"','') as grade_msg 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.result') end,'"','') as risk_result 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.rejected') end,'"','') as reject_tag 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.trace_id') end,'"','') as trace_id 
,r.mobile,r.id_card,r.data  
from db_credit.risk r  
where  r.data like '%免人审%'
# and  r.time > '2024-07-01 00:00:00'

;
'''
risk_data = query(sql)
risk_data.shape

In [ ]:
sql = '''-- 免人审 
select oa.* 
from db_credit.order_association oa 

;
'''
order_data = query(sql)
order_data.shape

In [ ]:
risk_data.sort_values(by=["trace_id","risk_date"],inplace=True)
risk_data.drop_duplicates(subset=["trace_id"],keep="last",inplace=True)
risk_data.shape

In [ ]:
ms_df = risk_data.merge(order_data,left_on='trace_id',right_on='risk_trace_id',how='left')
ms_df.shape

In [ ]:
ms_df.drop_duplicates(subset=["order_id"],keep="last",inplace=True)
ms_df.shape

In [ ]:
ms_df1 = df.merge(ms_df[['order_id','order_number','grade_msg']],on='order_id',how='left')
ms_df1.grade_msg.notnull().sum()

In [ ]:
df_tag4 = ms_df1[ms_df1.grade_msg.notnull()].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_ms = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_ms.to_excel(writer, sheet_name='mon_fstq_ms', index=True)  # index=False 表示不导出行索引  

In [ ]:
# 分周 
df_tag4 =ms_df1[ms_df1.grade_msg.notnull()].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_ms = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_ms.to_excel(writer, sheet_name='week_fstq_ms', index=True)  # index=False 表示不导出行索引  

# 分日

In [ ]:
df['if_当前逾期1+'] = np.where(df['当前逾期1+']=='逾期',1,0)

In [ ]:
# df_day_group.to_excel('F:/myfile/业务相关数据/日逾期数据导出240620.xlsx')

In [ ]:
# df_day_group = df[df.下单月份=='2024-06'].groupby(by=['下单日期']).agg({'order_id':'count','if_当前逾期1+':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum'})
df_day_group = df[df.下单日期>='2024-03-01 00:00:00'].groupby(by=['下单日期']).agg({'order_id':'count','if_当前逾期1+':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum'})
df_day_group = df_day_group.rename(columns={'order_id':'出库单数','if_当前逾期1+':'当前逾期1+'})

df_day_group['当前1+逾期率'] = (df_day_group['当前逾期1+']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd4_rate'] = (df_day_group['fpd4']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd7_rate'] = (df_day_group['fpd7']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd15_rate'] = (df_day_group['fpd15']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))

day_fstq = df_day_group[['出库单数','当前逾期1+','当前1+逾期率','fpd4','fpd4_rate','fpd7','fpd7_rate','fpd15','fpd15_rate']]

day_fstq

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    day_fstq.to_excel(writer, sheet_name='day_fstq', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_zm = df[df.归属渠道!='抖音渠道']
df_day_group = df_zm[df_zm.下单日期>='2024-03-01 00:00:00'].groupby(by=['下单日期']).agg({'order_id':'count','if_当前逾期1+':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum'})
df_day_group = df_day_group.rename(columns={'order_id':'出库单数','if_当前逾期1+':'当前逾期1+'})

df_day_group['当前1+逾期率'] = (df_day_group['当前逾期1+']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd4_rate'] = (df_day_group['fpd4']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd7_rate'] = (df_day_group['fpd7']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd15_rate'] = (df_day_group['fpd15']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))

day_fstq_nody = df_day_group[['出库单数','当前逾期1+','当前1+逾期率','fpd4','fpd4_rate','fpd7','fpd7_rate','fpd15','fpd15_rate']]

day_fstq_nody

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    day_fstq_nody.to_excel(writer, sheet_name='day_fstq_nody', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_zm = df[df.归属渠道=='芝麻租物']
df_day_group = df_zm[df_zm.下单日期>='2024-03-01 00:00:00'].groupby(by=['下单日期']).agg({'order_id':'count','if_当前逾期1+':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum'})
df_day_group = df_day_group.rename(columns={'order_id':'出库单数','if_当前逾期1+':'当前逾期1+'})

df_day_group['当前1+逾期率'] = (df_day_group['当前逾期1+']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd4_rate'] = (df_day_group['fpd4']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd7_rate'] = (df_day_group['fpd7']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd15_rate'] = (df_day_group['fpd15']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))

day_fstq_zm = df_day_group[['出库单数','当前逾期1+','当前1+逾期率','fpd4','fpd4_rate','fpd7','fpd7_rate','fpd15','fpd15_rate']]

day_fstq_zm

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    day_fstq_zm.to_excel(writer, sheet_name='day_fstq_zm', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_ss = df[df.归属渠道=='搜索渠道']
df_day_group = df_ss[df_ss.下单日期>='2024-03-01 00:00:00'].groupby(by=['下单日期']).agg({'order_id':'count','if_当前逾期1+':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum'})
df_day_group = df_day_group.rename(columns={'order_id':'出库单数','if_当前逾期1+':'当前逾期1+'})

df_day_group['当前1+逾期率'] = (df_day_group['当前逾期1+']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd4_rate'] = (df_day_group['fpd4']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd7_rate'] = (df_day_group['fpd7']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd15_rate'] = (df_day_group['fpd15']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))

day_fstq_ss = df_day_group[['出库单数','当前逾期1+','当前1+逾期率','fpd4','fpd4_rate','fpd7','fpd7_rate','fpd15','fpd15_rate']]

day_fstq_ss

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    day_fstq_ss.to_excel(writer, sheet_name='day_fstq_ss', index=True)  # index=False 表示不导出行索引  

In [ ]:
df_ss = df[df.归属渠道=='抖音渠道']
df_day_group = df_ss[df_ss.下单日期>='2024-03-01 00:00:00'].groupby(by=['下单日期']).agg({'order_id':'count','if_当前逾期1+':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum'})
df_day_group = df_day_group.rename(columns={'order_id':'出库单数','if_当前逾期1+':'当前逾期1+'})

df_day_group['当前1+逾期率'] = (df_day_group['当前逾期1+']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd4_rate'] = (df_day_group['fpd4']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd7_rate'] = (df_day_group['fpd7']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))
df_day_group['fpd15_rate'] = (df_day_group['fpd15']/df_day_group['出库单数']).apply(lambda x:format(x,".2%"))

day_fstq_dy = df_day_group[['出库单数','当前逾期1+','当前1+逾期率','fpd4','fpd4_rate','fpd7','fpd7_rate','fpd15','fpd15_rate']]

day_fstq_dy

In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    day_fstq_dy.to_excel(writer, sheet_name='day_fstq_dy', index=True)  # index=False 表示不导出行索引  

# 逾期4+ 还款期数分布

In [ ]:
df.columns

In [ ]:
back_term_df = pd.crosstab(df["下单月份"],df["已还期数"],margins=True)
back_term_df

In [ ]:
df_yq1 = df_yq[['订单号',	1,	2,	3,	4,	5,	6,	7,	8,	9,	10,	11,	12]]
df_yq1

In [ ]:
# 逾期天数大于等于4天的期数 
# df_yq1.columns = ['订单号'] + [str(x) for x in range(1, 13)]  
df_yq1['期数_逾期4+'] = df_yq1.iloc[:, 1:].apply(lambda row: sum(row >= 4), axis=1)  
df_yq1

In [ ]:
df_new =  df.merge(df_yq1[['订单号','期数_逾期4+']],on = '订单号',how='left')
df_new10 = df_new[df_new.已还期数>=10]
df_new11 = df_new[df_new.已还期数>=11]
df_new12 = df_new[df_new.已还期数>=12]


In [ ]:
overdue_term_df10 = pd.crosstab(df_new10["下单月份"],df_new10["期数_逾期4+"],margins=True)
overdue_term_df10

In [ ]:
overdue_term_df11 = pd.crosstab(df_new11["下单月份"],df_new11["期数_逾期4+"],margins=True)
overdue_term_df11

In [ ]:
overdue_term_df12 = pd.crosstab(df_new12["下单月份"],df_new12["期数_逾期4+"],margins=True)
overdue_term_df12

In [ ]:
df_new12.shape